In [1]:
import numpy as np
import torch 
import pandas as pd
import torchvision
from torch import optim,nn

In [2]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\train.csv',encoding='big5')
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2020\hw1\data\test.csv',encoding='big5',header=None)

In [3]:
train_data.shape

(4320, 27)

In [4]:
test_data.shape

(4320, 11)

In [5]:
train_data1=train_data.replace('NR',0)
train_data1=train_data1.iloc[:,3:]

In [6]:
train_data1.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
1,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5


- transform raw data to 18 dim per hours
- month data shape = 18,24 * 20

In [7]:
year_data={}
for m in range(12):
    month_data=np.empty((18,20*24))
    for d in range(20):
        month_data[:,24*d:24*(d+1)]=train_data1.iloc[18*(m*20+d):18*(m*20+d+1),:]
    year_data[m]=month_data

# try 1 hour group to predict No.2 data
- from 1 hours data to predict No.2 hour data
- 480 hour groups - 1 hour groups = 479 hour groups
- 479 hours data per month shape is 479,1 * 18

In [8]:
x=np.empty((12*479,1*18))
y=np.empty((12*479,1))

In [9]:
for m in range(12):
    for g in range(479):
        x[m*478+g:m*479+g+1,:]=year_data[m][:,g:g+1].reshape(1,-1)
        y[m*478+g:m*479+g+1,:]=year_data[m][9,g+1]

In [10]:
x1=pd.DataFrame(x)
x1=x1.apply(lambda x:(x-x.mean()) / x.std())
x1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-1.352925,0.777313,0.373559,0.566374,-0.542189,0.946894,0.624840,-0.848302,0.506742,0.275161,-0.09807,0.282412,-0.529992,0.878636,-1.246174,-1.303618,-0.842641,-1.140796
1,-1.352925,0.777313,0.064798,0.088675,-0.673552,-0.152184,-0.325436,-0.100758,0.277959,1.055293,-0.09807,-0.390786,-0.420008,0.878636,-0.797169,-0.839146,-0.467066,-0.764497
2,-1.352925,0.777313,0.003046,-0.102404,-0.717339,-0.313813,-0.470616,-0.260946,0.201698,0.875263,-0.09807,-0.465586,-0.584984,0.878636,-1.037335,-1.647750,-1.218215,-1.046722
3,-1.511777,0.777313,-0.058707,-0.197944,-0.191887,-0.523931,-0.483815,-0.474530,-0.293999,0.815253,-0.09807,0.058013,-0.639977,0.329022,-0.838937,-1.092494,-1.593789,-1.328946
4,-1.670628,0.777313,-0.120459,-0.293484,-0.148100,-0.540093,-0.497013,-0.421134,-0.675304,0.575212,-0.09807,-0.091587,-0.475000,0.329022,-0.483910,-0.680803,-0.560960,-1.046722


In [18]:
x1.shape

(5748, 18)

In [19]:
x1.corr()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.000000,-0.410253,-0.003671,0.081819,-0.022738,-0.131332,-0.114749,0.227978,0.086977,-0.015680,0.004118,-0.229093,0.288232,-0.224229,0.360197,0.339732,0.282557,0.369769
1,-0.410253,1.000000,0.127092,0.196142,0.101154,0.298571,0.274335,0.013451,0.219322,0.254007,-0.033389,0.000021,0.020642,0.777158,-0.069038,-0.082614,-0.222410,-0.258532
2,-0.003671,0.127092,1.000000,0.458692,0.256068,0.400970,0.404678,0.076235,0.273426,0.282795,-0.018411,-0.087703,0.364008,0.349718,0.079392,0.080579,0.004420,0.044909
3,0.081819,0.196142,0.458692,1.000000,0.445008,0.673011,0.683666,-0.047250,0.335591,0.291534,-0.009885,-0.055263,0.320850,0.709568,0.188806,0.174112,-0.064986,-0.023212
4,-0.022738,0.101154,0.256068,0.445008,1.000000,0.490764,0.702743,-0.232195,0.048047,0.029756,0.014053,0.012604,0.256500,0.323667,0.151833,0.142947,-0.074845,-0.074391
5,-0.131332,0.298571,0.400970,0.673011,0.490764,1.000000,0.963229,-0.102324,0.463476,0.449211,-0.021295,0.001097,0.427424,0.590806,0.216990,0.186595,-0.290004,-0.261596
6,-0.114749,0.274335,0.404678,0.683666,0.702743,0.963229,1.000000,-0.153149,0.392852,0.375580,-0.012075,0.004675,0.426264,0.579768,0.223253,0.196024,-0.259775,-0.236789
7,0.227978,0.013451,0.076235,-0.047250,-0.232195,-0.102324,-0.153149,1.000000,0.370834,0.356395,-0.032468,-0.671155,0.180364,-0.010667,0.278992,0.273017,0.300600,0.171060
8,0.086977,0.219322,0.273426,0.335591,0.048047,0.463476,0.392852,0.370834,1.000000,0.776192,-0.045975,-0.310977,0.401510,0.355271,0.228811,0.190725,-0.016344,0.007351
9,-0.015680,0.254007,0.282795,0.291534,0.029756,0.449211,0.375580,0.356395,0.776192,1.000000,-0.062575,-0.263686,0.371024,0.351444,0.186473,0.157257,-0.084923,-0.044975


In [11]:
def lin_reg(x,w,b):
    return torch.mm(x,w)+b

In [12]:
def mse_loss(y_hat,y):
    return (y_hat-y.view(y_hat.size()))**2

In [13]:
def sgd(params,lr,batch_size):
    for p in params:
        p.data -= lr * p.grad / batch_size

In [14]:
def train_hand(num_epochs,lr,train_freatues,train_labels,batch_size):
    dataset=torch.utils.data.TensorDataset(train_freatues,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for e in range(num_epochs):
        for x,y in data_iter:
            l=mse_loss(lin_reg(x,w,b),y).mean()
            
            if w.grad is not None:
                w.grad.data.zero_()
                b.grad.data.zero_()
            l.backward()
            sgd([w,b],lr,batch_size)
        if (e+1) %100 ==0:
            with torch.no_grad():
                print('epoch ',(e+1),'train loss : ',mse_loss(lin_reg(train_freatues,w,b),train_labels).mean().item())
            
        

In [15]:
x2=torch.Tensor(x1.values)
y2=torch.Tensor(y)
w=torch.Tensor(np.random.normal(0,0.01,(x2.shape[1],1)))
b=torch.Tensor([0.])
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)
lr=0.01
batch_size=256
train_hand(1000,lr,x2,y2,batch_size)

epoch  100 train loss :  454.3906555175781
epoch  200 train loss :  316.690673828125
epoch  300 train loss :  232.31495666503906
epoch  400 train loss :  176.11749267578125
epoch  500 train loss :  137.5991668701172
epoch  600 train loss :  110.83096313476562
epoch  700 train loss :  92.09595489501953
epoch  800 train loss :  78.8729476928711
epoch  900 train loss :  69.48177337646484
epoch  1000 train loss :  62.75888442993164


model by pytorch

In [16]:
def train_pytorch(num_epochs,lr,train_features,train_labels,batch_size):
    net=nn.Linear(train_features.shape[1],1)
    loss=nn.MSELoss()
    optimizer=optim.SGD(net.parameters(),lr=lr)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        if (e+1) %100==0:
            net.eval()
            with torch.no_grad():
                print('epoch ',(e+1),'train loss : ',loss(net(train_features),train_labels).item())
        

In [17]:
train_pytorch(1000,lr,x2,y2,batch_size)

epoch  100 train loss :  40.72090148925781
epoch  200 train loss :  40.721134185791016
epoch  300 train loss :  40.73210144042969
epoch  400 train loss :  40.71861267089844
epoch  500 train loss :  40.72087097167969
epoch  600 train loss :  40.7367057800293
epoch  700 train loss :  40.71577453613281
epoch  800 train loss :  40.71923828125
epoch  900 train loss :  40.722293853759766
epoch  1000 train loss :  40.718692779541016


- kfold to train model

In [18]:
def train_k_fold_data(net,num_epochs,lr,train_features,train_labels,test_features,test_labels,batch_size,montum,wd):
#     net=nn.Linear(train_features.shape[1],1)
#     net=nn.Sequential(nn.Linear(train_features.shape[1],1))
    loss=nn.MSELoss()
#     optimizer=optim.SGD(net.parameters(),lr=lr,momentum=montum,weight_decay=wd)
    optimizer=optim.Adam(net.parameters(),lr=lr,weight_decay=wd)
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    data_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    train_l,test_l=[],[]
    
    min_test_loss=1000
    early_stop_cnt=0
    train_loss,test_loss=0,0
    
    for e in range(num_epochs):
        net.train()
        for x,y in data_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#         if (e+1) %1000==0 and test_features is not  None:
        net.eval()
        with torch.no_grad():
            test_loss=loss(net(test_features),test_labels).item()
            if test_loss<min_test_loss:
                min_test_loss=test_loss
#                 test_l.append(test_loss)
                train_loss=loss(net(train_features),train_labels).item()
                test_l.append(test_loss)
                train_l.append(train_loss)
                print('epoch = %d train_loss : %f , test loss : %f' % (e+1,train_loss,test_loss))
                early_stop_cnt=0
            else:
                early_stop_cnt+=1
        if early_stop_cnt > 500:
            
            break
                
#             net.eval()
#             with torch.no_grad():
#                 train_l.append(loss(net(train_features),train_labels).item())
#                 test_l.append(loss(net(test_features),test_labels).item())
# #                 print('epoch ',(e+1),'train loss : ',train_l[-1],'test loss : ',test_l[-1])

    return train_l,test_l
        

In [19]:
def get_kfold_data(k,j,x,y):
    assert k>=1, 'k must >=1'
    fold_size = x.shape[0] // k
    x_train,y_train=None,None
    for i in range(k):
        idx=slice(fold_size*i,fold_size*(i+1))
        x_part,y_part=x[idx,:],y[idx,:]
        if i==j:
            x_val,y_val=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_val,y_val

In [20]:
def train_kfold(net1,num_epochs,lr,k,x_data,y_data,batch_size,montum,wd):
    train_sum_l,test_sum_l=[],[]
    train_l,test_l=[],[]
#     net=nn.Linear(x_data.shape[1])
    for j in range(k):
        net1=get_net()
#         for p in net1.parameters():
#             torch.nn.init.normal_(p)
        net=net1
        data=get_kfold_data(k,j,x_data,y_data)
        train_l,test_l=train_k_fold_data(net,num_epochs,lr,*data,batch_size,montum,wd)
        train_sum_l.append(train_l[-1])
        test_sum_l.append(test_l[-1])
        print('--------------------------------------------------------------------------')
        print('fold %d,train loss mean : %f,test loss : %f' % (j+1,train_l[-1],test_l[-1]))
        print('-------------------------------------------------------------------------')
    print('%d fold ,total train loss mean : %f,total test loss mean : %f ' % (k,np.mean(train_sum_l),np.mean(test_sum_l)))
    print('-----------------------------------------------------------------------------')

In [23]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],1))
net1=get_net()
train_kfold(net1,10000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 683.893066 , test loss : 972.142456
epoch = 2 train_loss : 680.320557 , test loss : 967.948853
epoch = 3 train_loss : 676.721741 , test loss : 963.661865
epoch = 4 train_loss : 673.281982 , test loss : 959.696594
epoch = 5 train_loss : 669.783386 , test loss : 955.702698
epoch = 6 train_loss : 666.354370 , test loss : 951.608887
epoch = 7 train_loss : 663.016235 , test loss : 947.764526
epoch = 8 train_loss : 659.697388 , test loss : 943.934021
epoch = 9 train_loss : 656.406799 , test loss : 940.235596
epoch = 10 train_loss : 653.222961 , test loss : 936.565735
epoch = 11 train_loss : 649.971436 , test loss : 932.730774
epoch = 12 train_loss : 646.877075 , test loss : 929.091309
epoch = 13 train_loss : 643.844299 , test loss : 925.831604
epoch = 14 train_loss : 640.796692 , test loss : 922.316650
epoch = 15 train_loss : 637.757507 , test loss : 918.701111
epoch = 16 train_loss : 634.802551 , test loss : 915.279541
epoch = 17 train_loss : 631.917114 , test loss : 

epoch = 141 train_loss : 432.050507 , test loss : 728.696777
epoch = 142 train_loss : 431.010773 , test loss : 727.864563
epoch = 143 train_loss : 429.972656 , test loss : 727.163208
epoch = 144 train_loss : 428.952087 , test loss : 726.574219
epoch = 145 train_loss : 427.939667 , test loss : 725.773499
epoch = 146 train_loss : 426.905029 , test loss : 725.111023
epoch = 147 train_loss : 425.897980 , test loss : 724.442078
epoch = 148 train_loss : 424.878967 , test loss : 723.780151
epoch = 149 train_loss : 423.871155 , test loss : 723.108826
epoch = 150 train_loss : 422.855408 , test loss : 722.154846
epoch = 151 train_loss : 421.849548 , test loss : 721.638977
epoch = 152 train_loss : 420.856720 , test loss : 720.804565
epoch = 153 train_loss : 419.858215 , test loss : 720.158203
epoch = 154 train_loss : 418.867615 , test loss : 719.422241
epoch = 155 train_loss : 417.862427 , test loss : 718.808655
epoch = 156 train_loss : 416.878876 , test loss : 718.096069
epoch = 157 train_loss :

epoch = 280 train_loss : 314.553589 , test loss : 640.051331
epoch = 281 train_loss : 313.871277 , test loss : 639.479980
epoch = 282 train_loss : 313.179840 , test loss : 638.900513
epoch = 283 train_loss : 312.489044 , test loss : 637.947693
epoch = 284 train_loss : 311.806213 , test loss : 637.484314
epoch = 285 train_loss : 311.127167 , test loss : 636.865356
epoch = 286 train_loss : 310.442688 , test loss : 636.135254
epoch = 287 train_loss : 309.768646 , test loss : 635.609802
epoch = 288 train_loss : 309.089264 , test loss : 635.009888
epoch = 289 train_loss : 308.420349 , test loss : 634.338501
epoch = 290 train_loss : 307.742828 , test loss : 633.730042
epoch = 291 train_loss : 307.072021 , test loss : 632.931885
epoch = 292 train_loss : 306.407166 , test loss : 632.484070
epoch = 293 train_loss : 305.740387 , test loss : 631.916321
epoch = 294 train_loss : 305.078033 , test loss : 631.277344
epoch = 295 train_loss : 304.407227 , test loss : 630.576355
epoch = 296 train_loss :

epoch = 415 train_loss : 235.788071 , test loss : 547.957581
epoch = 416 train_loss : 235.293167 , test loss : 547.132507
epoch = 417 train_loss : 234.804047 , test loss : 546.424744
epoch = 418 train_loss : 234.316788 , test loss : 545.658813
epoch = 419 train_loss : 233.829056 , test loss : 544.901611
epoch = 420 train_loss : 233.337265 , test loss : 544.081726
epoch = 421 train_loss : 232.848633 , test loss : 543.708923
epoch = 422 train_loss : 232.361862 , test loss : 542.605774
epoch = 423 train_loss : 231.876419 , test loss : 541.905334
epoch = 424 train_loss : 231.389755 , test loss : 540.912109
epoch = 425 train_loss : 230.910782 , test loss : 540.343323
epoch = 426 train_loss : 230.425415 , test loss : 539.542969
epoch = 427 train_loss : 229.943558 , test loss : 538.753113
epoch = 428 train_loss : 229.460892 , test loss : 538.085632
epoch = 429 train_loss : 228.983902 , test loss : 537.074341
epoch = 430 train_loss : 228.507431 , test loss : 536.418579
epoch = 431 train_loss :

epoch = 552 train_loss : 176.461975 , test loss : 431.310608
epoch = 553 train_loss : 176.082047 , test loss : 430.520142
epoch = 554 train_loss : 175.697342 , test loss : 429.577393
epoch = 555 train_loss : 175.314453 , test loss : 428.537903
epoch = 556 train_loss : 174.933701 , test loss : 427.612854
epoch = 557 train_loss : 174.551636 , test loss : 426.709717
epoch = 558 train_loss : 174.175873 , test loss : 425.898315
epoch = 559 train_loss : 173.794876 , test loss : 424.950836
epoch = 560 train_loss : 173.417725 , test loss : 424.085602
epoch = 561 train_loss : 173.035950 , test loss : 423.160980
epoch = 562 train_loss : 172.658951 , test loss : 422.308228
epoch = 563 train_loss : 172.281265 , test loss : 421.312500
epoch = 564 train_loss : 171.906586 , test loss : 420.443848
epoch = 565 train_loss : 171.532242 , test loss : 419.498505
epoch = 566 train_loss : 171.153931 , test loss : 418.520081
epoch = 567 train_loss : 170.783600 , test loss : 417.924835
epoch = 568 train_loss :

...

epoch = 781 train_loss : 110.388275 , test loss : 117.610130
epoch = 782 train_loss : 110.105820 , test loss : 117.302452
epoch = 783 train_loss : 109.819664 , test loss : 116.985008
epoch = 784 train_loss : 109.539719 , test loss : 116.687881
epoch = 785 train_loss : 109.256912 , test loss : 116.369339
epoch = 786 train_loss : 108.974686 , test loss : 116.058929
epoch = 787 train_loss : 108.695625 , test loss : 115.749817
epoch = 788 train_loss : 108.416679 , test loss : 115.472771
epoch = 789 train_loss : 108.136520 , test loss : 115.125664
epoch = 790 train_loss : 107.859314 , test loss : 114.840614
epoch = 791 train_loss : 107.580750 , test loss : 114.482323
epoch = 792 train_loss : 107.303383 , test loss : 114.203918
epoch = 793 train_loss : 107.026756 , test loss : 113.919533
epoch = 794 train_loss : 106.752579 , test loss : 113.600746
epoch = 795 train_loss : 106.475410 , test loss : 113.291275
epoch = 796 train_loss : 106.201492 , test loss : 113.010811
epoch = 797 train_loss :

epoch = 920 train_loss : 77.383942 , test loss : 80.661476
epoch = 921 train_loss : 77.189713 , test loss : 80.457008
epoch = 922 train_loss : 76.996284 , test loss : 80.228622
epoch = 923 train_loss : 76.805702 , test loss : 79.994110
epoch = 924 train_loss : 76.615929 , test loss : 79.806259
epoch = 925 train_loss : 76.428185 , test loss : 79.563293
epoch = 926 train_loss : 76.237381 , test loss : 79.358498
epoch = 927 train_loss : 76.047981 , test loss : 79.147049
epoch = 928 train_loss : 75.861496 , test loss : 78.931862
epoch = 929 train_loss : 75.672920 , test loss : 78.747238
epoch = 930 train_loss : 75.486549 , test loss : 78.513367
epoch = 931 train_loss : 75.301369 , test loss : 78.317429
epoch = 932 train_loss : 75.114250 , test loss : 78.074562
epoch = 933 train_loss : 74.929939 , test loss : 77.857590
epoch = 934 train_loss : 74.746346 , test loss : 77.672325
epoch = 935 train_loss : 74.561012 , test loss : 77.463127
epoch = 936 train_loss : 74.380058 , test loss : 77.2246

epoch = 1058 train_loss : 56.376732 , test loss : 56.283054
epoch = 1059 train_loss : 56.262115 , test loss : 56.148163
epoch = 1060 train_loss : 56.147747 , test loss : 56.028824
epoch = 1061 train_loss : 56.034733 , test loss : 55.894543
epoch = 1062 train_loss : 55.922886 , test loss : 55.750439
epoch = 1063 train_loss : 55.811562 , test loss : 55.640446
epoch = 1064 train_loss : 55.700180 , test loss : 55.490715
epoch = 1065 train_loss : 55.589111 , test loss : 55.351723
epoch = 1066 train_loss : 55.478718 , test loss : 55.221508
epoch = 1067 train_loss : 55.369678 , test loss : 55.109489
epoch = 1068 train_loss : 55.258369 , test loss : 54.976059
epoch = 1069 train_loss : 55.150909 , test loss : 54.848137
epoch = 1070 train_loss : 55.041435 , test loss : 54.725304
epoch = 1071 train_loss : 54.934883 , test loss : 54.588493
epoch = 1072 train_loss : 54.826622 , test loss : 54.464035
epoch = 1073 train_loss : 54.720768 , test loss : 54.354168
epoch = 1074 train_loss : 54.614880 , te

epoch = 1196 train_loss : 45.180264 , test loss : 43.315163
epoch = 1197 train_loss : 45.129486 , test loss : 43.253857
epoch = 1198 train_loss : 45.079063 , test loss : 43.200726
epoch = 1199 train_loss : 45.028866 , test loss : 43.143452
epoch = 1200 train_loss : 44.979568 , test loss : 43.091774
epoch = 1201 train_loss : 44.930904 , test loss : 43.033321
epoch = 1202 train_loss : 44.882736 , test loss : 42.983288
epoch = 1203 train_loss : 44.834557 , test loss : 42.942425
epoch = 1204 train_loss : 44.786842 , test loss : 42.882763
epoch = 1205 train_loss : 44.739090 , test loss : 42.835861
epoch = 1206 train_loss : 44.691662 , test loss : 42.779205
epoch = 1207 train_loss : 44.645622 , test loss : 42.736359
epoch = 1208 train_loss : 44.598846 , test loss : 42.678352
epoch = 1209 train_loss : 44.553440 , test loss : 42.627491
epoch = 1210 train_loss : 44.507431 , test loss : 42.580334
epoch = 1211 train_loss : 44.461597 , test loss : 42.533703
epoch = 1212 train_loss : 44.417034 , te

epoch = 1334 train_loss : 41.363590 , test loss : 39.680622
epoch = 1335 train_loss : 41.354527 , test loss : 39.675888
epoch = 1336 train_loss : 41.345131 , test loss : 39.672508
epoch = 1337 train_loss : 41.336571 , test loss : 39.670250
epoch = 1338 train_loss : 41.327206 , test loss : 39.668816
epoch = 1339 train_loss : 41.318432 , test loss : 39.662621
epoch = 1340 train_loss : 41.309723 , test loss : 39.661499
epoch = 1341 train_loss : 41.302139 , test loss : 39.653336
epoch = 1345 train_loss : 41.269550 , test loss : 39.649323
epoch = 1346 train_loss : 41.262493 , test loss : 39.645023
epoch = 1350 train_loss : 41.234367 , test loss : 39.644646
epoch = 1351 train_loss : 41.227436 , test loss : 39.643494
epoch = 1352 train_loss : 41.220325 , test loss : 39.640812
epoch = 1354 train_loss : 41.207890 , test loss : 39.636227
epoch = 1358 train_loss : 41.184055 , test loss : 39.634678
--------------------------------------------------------------------------
fold 5,train loss mean : 

In [26]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,5000,0.0001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 667.237793 , test loss : 958.723633
epoch = 2 train_loss : 659.427734 , test loss : 948.984131
epoch = 3 train_loss : 651.385620 , test loss : 938.762756
epoch = 4 train_loss : 643.002869 , test loss : 927.919922
epoch = 5 train_loss : 634.104919 , test loss : 916.334534
epoch = 6 train_loss : 624.521851 , test loss : 903.571655
epoch = 7 train_loss : 614.245483 , test loss : 889.898193
epoch = 8 train_loss : 603.326233 , test loss : 875.165344
epoch = 9 train_loss : 591.699524 , test loss : 859.326172
epoch = 10 train_loss : 579.542542 , test loss : 842.799194
epoch = 11 train_loss : 566.445862 , test loss : 824.926147
epoch = 12 train_loss : 552.962646 , test loss : 806.288574
epoch = 13 train_loss : 539.212524 , test loss : 787.450500
epoch = 14 train_loss : 524.706421 , test loss : 767.338318
epoch = 15 train_loss : 509.649200 , test loss : 746.389221
epoch = 16 train_loss : 494.699188 , test loss : 725.642334
epoch = 17 train_loss : 479.385223 , test loss : 

epoch = 141 train_loss : 83.291695 , test loss : 105.553856
epoch = 142 train_loss : 82.811661 , test loss : 105.157516
epoch = 143 train_loss : 82.346603 , test loss : 104.610275
epoch = 144 train_loss : 81.886368 , test loss : 103.803551
epoch = 145 train_loss : 81.422508 , test loss : 103.373299
epoch = 146 train_loss : 80.971138 , test loss : 102.887131
epoch = 147 train_loss : 80.523232 , test loss : 102.362854
epoch = 148 train_loss : 80.081886 , test loss : 101.618782
epoch = 149 train_loss : 79.636284 , test loss : 101.117996
epoch = 150 train_loss : 79.205147 , test loss : 100.601730
epoch = 151 train_loss : 78.772659 , test loss : 99.957939
epoch = 152 train_loss : 78.353569 , test loss : 99.548248
epoch = 153 train_loss : 77.926781 , test loss : 99.036194
epoch = 154 train_loss : 77.513519 , test loss : 98.413345
epoch = 155 train_loss : 77.100906 , test loss : 97.975952
epoch = 156 train_loss : 76.690788 , test loss : 97.318428
epoch = 157 train_loss : 76.290047 , test loss

epoch = 283 train_loss : 46.554863 , test loss : 59.501850
epoch = 284 train_loss : 46.419441 , test loss : 59.383011
epoch = 285 train_loss : 46.286674 , test loss : 59.205616
epoch = 286 train_loss : 46.154247 , test loss : 59.159962
epoch = 287 train_loss : 46.022804 , test loss : 58.959862
epoch = 288 train_loss : 45.892139 , test loss : 58.813278
epoch = 289 train_loss : 45.763123 , test loss : 58.657227
epoch = 290 train_loss : 45.631901 , test loss : 58.552723
epoch = 291 train_loss : 45.505520 , test loss : 58.416771
epoch = 292 train_loss : 45.377979 , test loss : 58.343147
epoch = 293 train_loss : 45.250931 , test loss : 58.212650
epoch = 294 train_loss : 45.129993 , test loss : 58.006119
epoch = 295 train_loss : 45.000683 , test loss : 57.939156
epoch = 296 train_loss : 44.878368 , test loss : 57.839764
epoch = 297 train_loss : 44.753864 , test loss : 57.678802
epoch = 298 train_loss : 44.636230 , test loss : 57.604252
epoch = 299 train_loss : 44.516895 , test loss : 57.4668

epoch = 428 train_loss : 34.258911 , test loss : 48.272694
epoch = 429 train_loss : 34.206100 , test loss : 48.231091
epoch = 430 train_loss : 34.153503 , test loss : 48.197052
epoch = 431 train_loss : 34.100567 , test loss : 48.161816
epoch = 432 train_loss : 34.048328 , test loss : 48.102390
epoch = 433 train_loss : 33.997818 , test loss : 48.050037
epoch = 434 train_loss : 33.947708 , test loss : 48.017723
epoch = 435 train_loss : 33.898537 , test loss : 48.002892
epoch = 436 train_loss : 33.844532 , test loss : 47.920509
epoch = 437 train_loss : 33.793888 , test loss : 47.911892
epoch = 438 train_loss : 33.742622 , test loss : 47.879616
epoch = 439 train_loss : 33.696007 , test loss : 47.801590
epoch = 440 train_loss : 33.653313 , test loss : 47.799744
epoch = 441 train_loss : 33.593430 , test loss : 47.755852
epoch = 442 train_loss : 33.541691 , test loss : 47.678738
epoch = 443 train_loss : 33.494465 , test loss : 47.676517
epoch = 444 train_loss : 33.446575 , test loss : 47.6525

epoch = 658 train_loss : 26.774101 , test loss : 43.344013
epoch = 667 train_loss : 26.599348 , test loss : 43.227497
epoch = 678 train_loss : 26.389797 , test loss : 43.189499
epoch = 682 train_loss : 26.315929 , test loss : 43.184826
epoch = 684 train_loss : 26.282852 , test loss : 43.161282
epoch = 688 train_loss : 26.208904 , test loss : 43.153206
epoch = 690 train_loss : 26.182632 , test loss : 43.126366
epoch = 694 train_loss : 26.107769 , test loss : 43.088135
epoch = 698 train_loss : 26.044744 , test loss : 43.058281
epoch = 701 train_loss : 25.991484 , test loss : 43.049644
epoch = 704 train_loss : 25.941795 , test loss : 43.025471
epoch = 709 train_loss : 25.858847 , test loss : 42.973705
epoch = 716 train_loss : 25.746227 , test loss : 42.916679
epoch = 724 train_loss : 25.619774 , test loss : 42.886894
epoch = 729 train_loss : 25.540237 , test loss : 42.872124
epoch = 731 train_loss : 25.513803 , test loss : 42.859249
epoch = 735 train_loss : 25.452375 , test loss : 42.8215

...

epoch = 286 train_loss : 52.297161 , test loss : 36.521328
epoch = 287 train_loss : 52.150276 , test loss : 36.500671
epoch = 289 train_loss : 51.860882 , test loss : 36.350086
epoch = 290 train_loss : 51.718075 , test loss : 36.260174
epoch = 292 train_loss : 51.435471 , test loss : 36.230415
epoch = 293 train_loss : 51.293762 , test loss : 36.178463
epoch = 294 train_loss : 51.163101 , test loss : 36.148766
epoch = 295 train_loss : 51.024612 , test loss : 36.052116
epoch = 296 train_loss : 50.887951 , test loss : 36.002151
epoch = 297 train_loss : 50.752998 , test loss : 35.853764
epoch = 300 train_loss : 50.358215 , test loss : 35.777000
epoch = 301 train_loss : 50.243576 , test loss : 35.636639
epoch = 305 train_loss : 49.718430 , test loss : 35.522541
epoch = 306 train_loss : 49.596638 , test loss : 35.516285
epoch = 308 train_loss : 49.358387 , test loss : 35.395069
epoch = 309 train_loss : 49.229591 , test loss : 35.347832
epoch = 310 train_loss : 49.111179 , test loss : 35.2382

epoch = 607 train_loss : 32.163040 , test loss : 28.430933
epoch = 613 train_loss : 32.013798 , test loss : 28.414289
epoch = 618 train_loss : 31.886209 , test loss : 28.381468
epoch = 619 train_loss : 31.862965 , test loss : 28.286768
epoch = 635 train_loss : 31.466616 , test loss : 28.283730
epoch = 638 train_loss : 31.398272 , test loss : 28.266417
epoch = 640 train_loss : 31.349510 , test loss : 28.222176
epoch = 649 train_loss : 31.144321 , test loss : 28.149424
epoch = 654 train_loss : 31.031437 , test loss : 28.138628
epoch = 658 train_loss : 30.945700 , test loss : 28.138584
epoch = 659 train_loss : 30.925100 , test loss : 28.045727
epoch = 660 train_loss : 30.904968 , test loss : 28.017624
epoch = 669 train_loss : 30.711105 , test loss : 28.012171
epoch = 673 train_loss : 30.632103 , test loss : 27.968519
epoch = 679 train_loss : 30.508934 , test loss : 27.958977
epoch = 698 train_loss : 30.129051 , test loss : 27.907845
epoch = 699 train_loss : 30.110468 , test loss : 27.8837

epoch = 109 train_loss : 100.353905 , test loss : 137.619995
epoch = 110 train_loss : 99.751785 , test loss : 137.327850
epoch = 111 train_loss : 99.178001 , test loss : 137.092300
epoch = 112 train_loss : 98.584465 , test loss : 136.781677
epoch = 113 train_loss : 97.990181 , test loss : 136.426895
epoch = 114 train_loss : 97.427338 , test loss : 136.183167
epoch = 115 train_loss : 96.850388 , test loss : 135.879318
epoch = 116 train_loss : 96.293198 , test loss : 135.583618
epoch = 117 train_loss : 95.733009 , test loss : 135.237289
epoch = 118 train_loss : 95.188919 , test loss : 134.891541
epoch = 119 train_loss : 94.663818 , test loss : 134.546829
epoch = 120 train_loss : 94.121033 , test loss : 134.179443
epoch = 121 train_loss : 93.592293 , test loss : 133.817184
epoch = 122 train_loss : 93.064217 , test loss : 133.502182
epoch = 123 train_loss : 92.565453 , test loss : 133.082397
epoch = 124 train_loss : 92.037163 , test loss : 132.666855
epoch = 125 train_loss : 91.536781 , te

epoch = 249 train_loss : 53.561558 , test loss : 84.546761
epoch = 250 train_loss : 53.385883 , test loss : 84.299782
epoch = 251 train_loss : 53.223145 , test loss : 83.938522
epoch = 252 train_loss : 53.054474 , test loss : 83.707077
epoch = 253 train_loss : 52.886669 , test loss : 83.498825
epoch = 254 train_loss : 52.725277 , test loss : 83.182732
epoch = 256 train_loss : 52.406139 , test loss : 82.682770
epoch = 257 train_loss : 52.237728 , test loss : 82.493530
epoch = 258 train_loss : 52.075745 , test loss : 82.359879
epoch = 259 train_loss : 51.920128 , test loss : 82.039833
epoch = 260 train_loss : 51.769459 , test loss : 81.925705
epoch = 261 train_loss : 51.608719 , test loss : 81.631790
epoch = 262 train_loss : 51.453388 , test loss : 81.409386
epoch = 263 train_loss : 51.301754 , test loss : 81.149559
epoch = 264 train_loss : 51.151020 , test loss : 80.915482
epoch = 265 train_loss : 51.000427 , test loss : 80.748283
epoch = 266 train_loss : 50.852497 , test loss : 80.6357

epoch = 423 train_loss : 36.944553 , test loss : 62.105370
epoch = 424 train_loss : 36.893837 , test loss : 61.986877
epoch = 426 train_loss : 36.789688 , test loss : 61.944397
epoch = 428 train_loss : 36.683922 , test loss : 61.827072
epoch = 430 train_loss : 36.582165 , test loss : 61.771439
epoch = 432 train_loss : 36.479797 , test loss : 61.642765
epoch = 433 train_loss : 36.429588 , test loss : 61.567013
epoch = 435 train_loss : 36.328381 , test loss : 61.413086
epoch = 436 train_loss : 36.284035 , test loss : 61.382778
epoch = 438 train_loss : 36.180332 , test loss : 61.271622
epoch = 439 train_loss : 36.134205 , test loss : 61.172340
epoch = 441 train_loss : 36.041996 , test loss : 61.047005
epoch = 444 train_loss : 35.894917 , test loss : 60.948299
epoch = 445 train_loss : 35.845955 , test loss : 60.942814
epoch = 446 train_loss : 35.800598 , test loss : 60.860222
epoch = 447 train_loss : 35.754745 , test loss : 60.829189
epoch = 448 train_loss : 35.706875 , test loss : 60.8133

In [44]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 566.760864 , test loss : 843.992920
epoch = 2 train_loss : 419.449768 , test loss : 641.294006
epoch = 3 train_loss : 277.096375 , test loss : 431.870422
epoch = 4 train_loss : 205.407700 , test loss : 309.336761
epoch = 5 train_loss : 168.469940 , test loss : 248.702408
epoch = 6 train_loss : 141.647049 , test loss : 204.040497
epoch = 7 train_loss : 124.621384 , test loss : 172.370636
epoch = 8 train_loss : 113.505768 , test loss : 151.599594
epoch = 9 train_loss : 105.225273 , test loss : 141.329483
epoch = 10 train_loss : 98.320915 , test loss : 132.120621
epoch = 11 train_loss : 91.822044 , test loss : 123.262085
epoch = 12 train_loss : 86.247261 , test loss : 118.440132
epoch = 13 train_loss : 81.264191 , test loss : 109.368713
epoch = 14 train_loss : 76.761879 , test loss : 104.390289
epoch = 15 train_loss : 72.969254 , test loss : 99.610245
epoch = 16 train_loss : 69.929527 , test loss : 91.859383
epoch = 17 train_loss : 66.897942 , test loss : 90.842735


epoch = 91 train_loss : 20.751526 , test loss : 54.648636
epoch = 99 train_loss : 19.653423 , test loss : 53.929810
epoch = 111 train_loss : 18.201227 , test loss : 53.698154
epoch = 130 train_loss : 16.200338 , test loss : 53.535374
--------------------------------------------------------------------------
fold 2,train loss mean : 16.200338,test loss : 53.535374
-------------------------------------------------------------------------
epoch = 1 train_loss : 653.878479 , test loss : 482.925537
epoch = 2 train_loss : 467.844482 , test loss : 441.050201
epoch = 3 train_loss : 308.002716 , test loss : 392.221741
epoch = 4 train_loss : 238.060150 , test loss : 349.436554
epoch = 5 train_loss : 189.171707 , test loss : 312.276001
epoch = 6 train_loss : 156.561584 , test loss : 269.284607
epoch = 7 train_loss : 139.959473 , test loss : 232.022827
epoch = 8 train_loss : 126.850792 , test loss : 202.703720
epoch = 9 train_loss : 115.048225 , test loss : 175.392624
epoch = 10 train_loss : 104.6

epoch = 8 train_loss : 114.662621 , test loss : 155.833115
epoch = 9 train_loss : 105.681732 , test loss : 153.612274
epoch = 10 train_loss : 98.707565 , test loss : 150.377808
epoch = 11 train_loss : 92.689857 , test loss : 146.106781
epoch = 12 train_loss : 87.656334 , test loss : 141.459900
epoch = 13 train_loss : 83.064644 , test loss : 134.835190
epoch = 14 train_loss : 79.124405 , test loss : 128.818710
epoch = 15 train_loss : 75.679390 , test loss : 123.615692
epoch = 16 train_loss : 72.565781 , test loss : 118.267723
epoch = 17 train_loss : 69.719307 , test loss : 114.187798
epoch = 18 train_loss : 67.498131 , test loss : 109.952377
epoch = 19 train_loss : 64.950348 , test loss : 106.955383
epoch = 20 train_loss : 63.010998 , test loss : 104.323303
epoch = 21 train_loss : 61.024269 , test loss : 100.730034
epoch = 22 train_loss : 59.227325 , test loss : 99.096970
epoch = 23 train_loss : 57.478962 , test loss : 96.529427
epoch = 24 train_loss : 55.988064 , test loss : 93.924713


In [45]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],256),nn.ReLU(),nn.Linear(256,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 489.560944 , test loss : 737.100891
epoch = 2 train_loss : 286.326019 , test loss : 439.160767
epoch = 3 train_loss : 195.855240 , test loss : 280.635315
epoch = 4 train_loss : 149.335785 , test loss : 209.671539
epoch = 5 train_loss : 123.795982 , test loss : 168.024094
epoch = 6 train_loss : 109.883774 , test loss : 140.009995
epoch = 7 train_loss : 99.993637 , test loss : 129.912064
epoch = 8 train_loss : 91.554131 , test loss : 122.804520
epoch = 9 train_loss : 84.496735 , test loss : 111.665627
epoch = 10 train_loss : 78.659096 , test loss : 103.608253
epoch = 11 train_loss : 73.692909 , test loss : 98.248123
epoch = 12 train_loss : 69.421860 , test loss : 91.341866
epoch = 13 train_loss : 65.982079 , test loss : 87.039726
epoch = 14 train_loss : 63.038349 , test loss : 83.681274
epoch = 15 train_loss : 60.373013 , test loss : 79.002037
epoch = 16 train_loss : 58.230846 , test loss : 76.340706
epoch = 17 train_loss : 55.865303 , test loss : 75.266121
epoch =

epoch = 31 train_loss : 36.432945 , test loss : 68.577347
epoch = 32 train_loss : 35.796738 , test loss : 68.045235
epoch = 33 train_loss : 35.023281 , test loss : 66.500450
epoch = 35 train_loss : 33.396080 , test loss : 64.928787
epoch = 37 train_loss : 32.281189 , test loss : 64.271179
epoch = 38 train_loss : 31.590775 , test loss : 63.007275
epoch = 39 train_loss : 31.048189 , test loss : 62.467838
epoch = 41 train_loss : 29.949213 , test loss : 61.096180
epoch = 44 train_loss : 28.389158 , test loss : 59.959675
epoch = 45 train_loss : 28.360891 , test loss : 59.340710
epoch = 47 train_loss : 27.188028 , test loss : 58.729961
epoch = 48 train_loss : 27.383219 , test loss : 58.042835
epoch = 50 train_loss : 26.583067 , test loss : 57.752430
epoch = 51 train_loss : 26.123472 , test loss : 57.663982
epoch = 52 train_loss : 25.485373 , test loss : 57.235558
epoch = 53 train_loss : 25.533976 , test loss : 56.959881
epoch = 54 train_loss : 25.006058 , test loss : 56.504639
epoch = 56 tra

In [46]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.ReLU(),nn.Linear(512,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 431.102905 , test loss : 665.350159
epoch = 2 train_loss : 212.281403 , test loss : 319.986267
epoch = 3 train_loss : 151.192200 , test loss : 208.601471
epoch = 4 train_loss : 118.835556 , test loss : 159.885391
epoch = 5 train_loss : 103.302490 , test loss : 131.199265
epoch = 6 train_loss : 92.132278 , test loss : 121.132614
epoch = 7 train_loss : 83.795204 , test loss : 109.735023
epoch = 8 train_loss : 76.748207 , test loss : 98.939819
epoch = 9 train_loss : 71.171326 , test loss : 92.993576
epoch = 10 train_loss : 66.595055 , test loss : 86.388390
epoch = 11 train_loss : 62.938557 , test loss : 82.025986
epoch = 12 train_loss : 59.518353 , test loss : 78.547325
epoch = 13 train_loss : 56.537624 , test loss : 73.634491
epoch = 14 train_loss : 53.814808 , test loss : 70.155495
epoch = 16 train_loss : 48.872410 , test loss : 64.432320
epoch = 17 train_loss : 46.980022 , test loss : 62.459698
epoch = 18 train_loss : 44.667858 , test loss : 60.474670
epoch = 19 

--------------------------------------------------------------------------
fold 3,train loss mean : 17.056786,test loss : 51.066326
-------------------------------------------------------------------------
epoch = 1 train_loss : 471.346832 , test loss : 294.336456
epoch = 2 train_loss : 244.959839 , test loss : 190.207916
epoch = 3 train_loss : 167.726685 , test loss : 120.876175
epoch = 4 train_loss : 139.978058 , test loss : 96.316124
epoch = 5 train_loss : 122.542542 , test loss : 74.523468
epoch = 6 train_loss : 108.455750 , test loss : 59.253708
epoch = 7 train_loss : 97.212669 , test loss : 50.887413
epoch = 8 train_loss : 88.455421 , test loss : 46.832981
epoch = 9 train_loss : 81.804108 , test loss : 45.875809
epoch = 10 train_loss : 76.634392 , test loss : 43.115360
epoch = 11 train_loss : 71.667236 , test loss : 41.794418
epoch = 12 train_loss : 67.571846 , test loss : 41.786388
epoch = 13 train_loss : 64.592735 , test loss : 40.661888
epoch = 14 train_loss : 61.246471 , test

In [48]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.ReLU(),nn.Linear(512,256),nn.ReLU(),nn.Linear(256,1),nn.ReLU())
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 177.848618 , test loss : 233.494217
epoch = 2 train_loss : 109.186966 , test loss : 145.820679
epoch = 3 train_loss : 83.520164 , test loss : 108.172401
epoch = 4 train_loss : 71.958801 , test loss : 90.195412
epoch = 5 train_loss : 65.074226 , test loss : 84.377945
epoch = 6 train_loss : 59.607510 , test loss : 75.991150
epoch = 7 train_loss : 54.984627 , test loss : 70.223198
epoch = 8 train_loss : 51.654339 , test loss : 66.309669
epoch = 9 train_loss : 48.474545 , test loss : 65.186264
epoch = 10 train_loss : 45.143303 , test loss : 58.778099
epoch = 11 train_loss : 42.446018 , test loss : 56.778885
epoch = 12 train_loss : 39.826942 , test loss : 54.296951
epoch = 13 train_loss : 37.750511 , test loss : 51.609367
epoch = 14 train_loss : 35.790749 , test loss : 50.015728
epoch = 15 train_loss : 33.508759 , test loss : 48.502369
epoch = 16 train_loss : 31.782032 , test loss : 47.274750
epoch = 17 train_loss : 30.004538 , test loss : 46.633724
epoch = 18 train_l

In [49]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.ReLU(),nn.Linear(512,256),nn.ReLU(),nn.Linear(256,64),nn.ReLU(),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 203.917038 , test loss : 219.193192
epoch = 2 train_loss : 102.257050 , test loss : 114.416130
epoch = 3 train_loss : 81.945557 , test loss : 108.360283
epoch = 4 train_loss : 70.236931 , test loss : 86.203583
epoch = 5 train_loss : 64.371040 , test loss : 77.147026
epoch = 6 train_loss : 57.360935 , test loss : 72.474556
epoch = 7 train_loss : 53.182045 , test loss : 69.270187
epoch = 8 train_loss : 48.483650 , test loss : 62.548161
epoch = 9 train_loss : 44.808426 , test loss : 60.184456
epoch = 10 train_loss : 40.506908 , test loss : 54.192776
epoch = 11 train_loss : 37.456078 , test loss : 51.925629
epoch = 12 train_loss : 34.792366 , test loss : 49.209072
epoch = 13 train_loss : 31.829411 , test loss : 48.150475
epoch = 15 train_loss : 28.608433 , test loss : 46.667446
epoch = 19 train_loss : 23.643719 , test loss : 45.068569
epoch = 23 train_loss : 19.597992 , test loss : 44.904518
--------------------------------------------------------------------------
f

In [50]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.Linear(512,256),nn.Linear(256,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 370.200989 , test loss : 975.947144
epoch = 2 train_loss : 119.004456 , test loss : 225.845840
epoch = 3 train_loss : 56.284367 , test loss : 68.936760
epoch = 4 train_loss : 44.133900 , test loss : 49.682083
epoch = 5 train_loss : 38.886478 , test loss : 46.141094
epoch = 7 train_loss : 36.623833 , test loss : 40.103817
epoch = 24 train_loss : 31.775635 , test loss : 39.962753
epoch = 34 train_loss : 32.232460 , test loss : 39.764481
epoch = 61 train_loss : 32.892273 , test loss : 39.207092
epoch = 131 train_loss : 32.400681 , test loss : 39.031471
epoch = 225 train_loss : 32.038151 , test loss : 39.023426
epoch = 245 train_loss : 31.589241 , test loss : 38.841141
--------------------------------------------------------------------------
fold 1,train loss mean : 31.589241,test loss : 38.841141
-------------------------------------------------------------------------
epoch = 1 train_loss : 348.001801 , test loss : 589.714355
epoch = 2 train_loss : 78.007263 , tes

In [51]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.Linear(512,256),nn.Linear(256,128),nn.Linear(128,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 2 train_loss : 70.794411 , test loss : 94.551636
epoch = 3 train_loss : 51.006752 , test loss : 57.303684
epoch = 4 train_loss : 39.236267 , test loss : 46.117249
epoch = 8 train_loss : 36.674694 , test loss : 42.481285
epoch = 9 train_loss : 34.638115 , test loss : 41.427792
epoch = 10 train_loss : 34.138569 , test loss : 41.157822
epoch = 17 train_loss : 33.381920 , test loss : 40.734509
epoch = 19 train_loss : 33.494488 , test loss : 40.146076
epoch = 21 train_loss : 32.869373 , test loss : 39.311230
epoch = 31 train_loss : 32.973263 , test loss : 38.931908
--------------------------------------------------------------------------
fold 1,train loss mean : 32.973263,test loss : 38.931908
-------------------------------------------------------------------------
epoch = 1 train_loss : 343.270599 , test loss : 670.568726
epoch = 2 train_loss : 64.780464 , test loss : 86.424179
epoch = 3 train_loss : 45.525188 , test loss : 58.341770
epoch = 4 train_loss : 39.795208 , test loss :

In [58]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.Linear(512,256),nn.Linear(256,128),nn.Linear(128,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,3000,0.00001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 667.645325 , test loss : 969.028320
epoch = 2 train_loss : 665.386902 , test loss : 966.190430
epoch = 3 train_loss : 662.917175 , test loss : 962.971375
epoch = 4 train_loss : 660.167297 , test loss : 959.350952
epoch = 5 train_loss : 656.992065 , test loss : 955.155151
epoch = 6 train_loss : 653.166077 , test loss : 949.987610
epoch = 7 train_loss : 648.680786 , test loss : 943.897278
epoch = 8 train_loss : 643.373047 , test loss : 937.191040
epoch = 9 train_loss : 637.060181 , test loss : 929.073486
epoch = 10 train_loss : 629.525146 , test loss : 919.172058
epoch = 11 train_loss : 620.531494 , test loss : 907.282104
epoch = 12 train_loss : 610.451904 , test loss : 894.567871
epoch = 13 train_loss : 599.044495 , test loss : 879.843933
epoch = 14 train_loss : 586.514038 , test loss : 864.799744
epoch = 15 train_loss : 573.444031 , test loss : 849.561707
epoch = 16 train_loss : 559.499390 , test loss : 833.663879
epoch = 17 train_loss : 546.312073 , test loss : 

epoch = 291 train_loss : 31.920427 , test loss : 40.301582
epoch = 298 train_loss : 31.853418 , test loss : 40.272678
epoch = 309 train_loss : 31.807878 , test loss : 40.195995
epoch = 311 train_loss : 31.808655 , test loss : 40.180199
epoch = 314 train_loss : 31.701973 , test loss : 40.137714
epoch = 326 train_loss : 31.626600 , test loss : 40.117496
epoch = 327 train_loss : 31.618784 , test loss : 40.024296
epoch = 342 train_loss : 31.544466 , test loss : 39.977684
epoch = 359 train_loss : 31.430740 , test loss : 39.901115
epoch = 366 train_loss : 31.408232 , test loss : 39.867100
epoch = 373 train_loss : 31.395575 , test loss : 39.760948
epoch = 405 train_loss : 31.280840 , test loss : 39.718441
epoch = 467 train_loss : 31.208303 , test loss : 39.592751
epoch = 575 train_loss : 31.020260 , test loss : 39.566380
epoch = 676 train_loss : 30.947969 , test loss : 39.564499
--------------------------------------------------------------------------
fold 1,train loss mean : 30.947969,test 

epoch = 135 train_loss : 35.653305 , test loss : 47.262539
epoch = 136 train_loss : 35.587849 , test loss : 47.049061
epoch = 138 train_loss : 35.469894 , test loss : 46.949726
epoch = 139 train_loss : 35.410877 , test loss : 46.913712
epoch = 140 train_loss : 35.353214 , test loss : 46.722267
epoch = 141 train_loss : 35.303719 , test loss : 46.558350
epoch = 142 train_loss : 35.245293 , test loss : 46.532528
epoch = 143 train_loss : 35.190018 , test loss : 46.452160
epoch = 144 train_loss : 35.135418 , test loss : 46.363243
epoch = 145 train_loss : 35.084824 , test loss : 46.325420
epoch = 146 train_loss : 35.043091 , test loss : 46.265594
epoch = 147 train_loss : 34.979836 , test loss : 46.051079
epoch = 149 train_loss : 34.880615 , test loss : 45.969559
epoch = 150 train_loss : 34.835644 , test loss : 45.872795
epoch = 151 train_loss : 34.785072 , test loss : 45.806904
epoch = 152 train_loss : 34.741146 , test loss : 45.745266
epoch = 153 train_loss : 34.697800 , test loss : 45.6464

epoch = 78 train_loss : 102.960800 , test loss : 254.591736
epoch = 79 train_loss : 96.081230 , test loss : 228.779617
epoch = 80 train_loss : 89.663338 , test loss : 209.984192
epoch = 81 train_loss : 83.835518 , test loss : 190.599747
epoch = 82 train_loss : 78.435349 , test loss : 173.845673
epoch = 83 train_loss : 73.599380 , test loss : 159.919754
epoch = 84 train_loss : 69.237717 , test loss : 145.540237
epoch = 85 train_loss : 65.348122 , test loss : 132.825623
epoch = 86 train_loss : 61.941986 , test loss : 122.295143
epoch = 87 train_loss : 58.967579 , test loss : 111.072807
epoch = 88 train_loss : 56.353497 , test loss : 103.160156
epoch = 89 train_loss : 54.102776 , test loss : 97.227371
epoch = 90 train_loss : 52.141518 , test loss : 90.313118
epoch = 91 train_loss : 50.444569 , test loss : 83.413338
epoch = 92 train_loss : 49.029350 , test loss : 79.772270
epoch = 93 train_loss : 47.729195 , test loss : 75.713623
epoch = 94 train_loss : 46.649612 , test loss : 72.268013
ep

epoch = 322 train_loss : 31.176117 , test loss : 43.623020
epoch = 333 train_loss : 31.090927 , test loss : 43.562851
epoch = 338 train_loss : 31.060230 , test loss : 43.535381
epoch = 341 train_loss : 31.089647 , test loss : 43.456123
epoch = 351 train_loss : 31.081182 , test loss : 43.435131
epoch = 356 train_loss : 30.969004 , test loss : 43.413319
epoch = 357 train_loss : 30.965218 , test loss : 43.407337
epoch = 360 train_loss : 30.968838 , test loss : 43.397034
epoch = 362 train_loss : 30.945221 , test loss : 43.329239
epoch = 370 train_loss : 30.916897 , test loss : 43.282135
epoch = 376 train_loss : 30.900656 , test loss : 43.239956
epoch = 380 train_loss : 30.877872 , test loss : 43.202232
epoch = 382 train_loss : 30.935303 , test loss : 43.121620
epoch = 391 train_loss : 30.848686 , test loss : 43.119820
epoch = 404 train_loss : 30.814976 , test loss : 43.049248
epoch = 412 train_loss : 30.766516 , test loss : 43.041222
epoch = 414 train_loss : 30.760668 , test loss : 43.0409

epoch = 187 train_loss : 38.472961 , test loss : 23.071943
epoch = 196 train_loss : 38.208378 , test loss : 23.009892
epoch = 199 train_loss : 38.084274 , test loss : 22.977615
epoch = 203 train_loss : 37.937389 , test loss : 22.977407
epoch = 207 train_loss : 37.851101 , test loss : 22.939880
epoch = 212 train_loss : 37.680168 , test loss : 22.890139
epoch = 228 train_loss : 37.321339 , test loss : 22.840845
epoch = 234 train_loss : 37.185059 , test loss : 22.780117
epoch = 243 train_loss : 37.018280 , test loss : 22.746166
epoch = 245 train_loss : 37.000481 , test loss : 22.728249
epoch = 282 train_loss : 36.441086 , test loss : 22.708366
epoch = 304 train_loss : 36.206722 , test loss : 22.703358
epoch = 320 train_loss : 36.077354 , test loss : 22.618937
epoch = 341 train_loss : 35.917809 , test loss : 22.595045
epoch = 398 train_loss : 35.622551 , test loss : 22.562904
epoch = 410 train_loss : 35.570698 , test loss : 22.552895
epoch = 486 train_loss : 35.384689 , test loss : 22.4658

epoch = 121 train_loss : 41.163013 , test loss : 45.131725
epoch = 123 train_loss : 40.902962 , test loss : 44.760582
epoch = 124 train_loss : 40.771702 , test loss : 44.607159
epoch = 126 train_loss : 40.527893 , test loss : 44.180634
epoch = 128 train_loss : 40.322701 , test loss : 43.687153
epoch = 130 train_loss : 40.064899 , test loss : 43.539253
epoch = 131 train_loss : 39.957932 , test loss : 43.425068
epoch = 132 train_loss : 39.855736 , test loss : 43.297016
epoch = 134 train_loss : 39.647541 , test loss : 43.030849
epoch = 135 train_loss : 39.546310 , test loss : 42.813320
epoch = 136 train_loss : 39.448551 , test loss : 42.648361
epoch = 137 train_loss : 39.353130 , test loss : 42.586765
epoch = 138 train_loss : 39.261620 , test loss : 42.452732
epoch = 139 train_loss : 39.167500 , test loss : 42.315880
epoch = 140 train_loss : 39.098717 , test loss : 42.267639
epoch = 141 train_loss : 39.008965 , test loss : 41.995396
epoch = 143 train_loss : 38.838436 , test loss : 41.8282

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.Linear(512,256),nn.Linear(256,128),nn.Linear(128,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.0001,6,x2,y2,256,0,0)

epoch = 2 train_loss : 559.470642 , test loss : 917.679443
epoch = 3 train_loss : 491.707916 , test loss : 874.093079
epoch = 6 train_loss : 262.260223 , test loss : 734.933838
epoch = 7 train_loss : 145.849777 , test loss : 418.768494
epoch = 8 train_loss : 66.214508 , test loss : 152.250000
epoch = 9 train_loss : 46.360500 , test loss : 56.398663
epoch = 10 train_loss : 42.174145 , test loss : 55.903873
epoch = 11 train_loss : 40.284676 , test loss : 50.979027
epoch = 12 train_loss : 40.320271 , test loss : 49.795532
epoch = 13 train_loss : 37.809826 , test loss : 49.136669
epoch = 14 train_loss : 37.995686 , test loss : 48.228012
epoch = 16 train_loss : 36.494122 , test loss : 45.283875
epoch = 17 train_loss : 35.097103 , test loss : 45.225262
epoch = 19 train_loss : 34.418335 , test loss : 44.269138
epoch = 21 train_loss : 33.683632 , test loss : 43.672504
epoch = 32 train_loss : 32.326977 , test loss : 42.580250
epoch = 59 train_loss : 31.296209 , test loss : 41.843212
epoch = 220

In [ ]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],512),nn.Linear(512,256),nn.Linear(256,128),nn.Linear(128,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.0001,1,x2,y2,256,0,0)

In [33]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],64),nn.ReLU(),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,3000,0.0001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 658.516357 , test loss : 957.684326
epoch = 2 train_loss : 639.375183 , test loss : 934.808472
epoch = 3 train_loss : 616.310303 , test loss : 905.893738
epoch = 4 train_loss : 586.110840 , test loss : 866.125305
epoch = 5 train_loss : 545.752075 , test loss : 810.706177
epoch = 6 train_loss : 493.771698 , test loss : 737.030884
epoch = 7 train_loss : 432.430878 , test loss : 647.581604
epoch = 8 train_loss : 369.208221 , test loss : 552.371704
epoch = 9 train_loss : 314.247803 , test loss : 466.504944
epoch = 10 train_loss : 273.426910 , test loss : 399.694244
epoch = 11 train_loss : 246.268738 , test loss : 353.455811
epoch = 12 train_loss : 228.252274 , test loss : 323.305695
epoch = 13 train_loss : 214.612839 , test loss : 301.583069
epoch = 14 train_loss : 203.334305 , test loss : 285.949127
epoch = 15 train_loss : 193.064529 , test loss : 272.514435
epoch = 16 train_loss : 183.418777 , test loss : 259.869507
epoch = 17 train_loss : 174.414749 , test loss : 

epoch = 141 train_loss : 47.866295 , test loss : 63.330479
epoch = 142 train_loss : 47.727703 , test loss : 62.999275
epoch = 143 train_loss : 47.593475 , test loss : 62.804592
epoch = 144 train_loss : 47.460697 , test loss : 62.708748
epoch = 145 train_loss : 47.329575 , test loss : 62.531723
epoch = 146 train_loss : 47.198143 , test loss : 62.305710
epoch = 147 train_loss : 47.070206 , test loss : 61.988129
epoch = 148 train_loss : 46.942558 , test loss : 61.872562
epoch = 149 train_loss : 46.818710 , test loss : 61.638958
epoch = 150 train_loss : 46.695251 , test loss : 61.428261
epoch = 151 train_loss : 46.570320 , test loss : 61.339809
epoch = 152 train_loss : 46.449265 , test loss : 61.156849
epoch = 153 train_loss : 46.329395 , test loss : 61.005684
epoch = 154 train_loss : 46.210510 , test loss : 60.859337
epoch = 155 train_loss : 46.092403 , test loss : 60.712902
epoch = 156 train_loss : 45.975460 , test loss : 60.659019
epoch = 157 train_loss : 45.862263 , test loss : 60.5638

epoch = 295 train_loss : 36.325989 , test loss : 48.208984
epoch = 296 train_loss : 36.283543 , test loss : 48.158478
epoch = 297 train_loss : 36.244675 , test loss : 48.015762
epoch = 298 train_loss : 36.204288 , test loss : 47.960621
epoch = 299 train_loss : 36.163689 , test loss : 47.901466
epoch = 301 train_loss : 36.083126 , test loss : 47.825527
epoch = 302 train_loss : 36.042686 , test loss : 47.793747
epoch = 304 train_loss : 35.961536 , test loss : 47.767113
epoch = 305 train_loss : 35.922951 , test loss : 47.697838
epoch = 306 train_loss : 35.883717 , test loss : 47.672562
epoch = 307 train_loss : 35.844635 , test loss : 47.644543
epoch = 308 train_loss : 35.806309 , test loss : 47.581829
epoch = 309 train_loss : 35.768127 , test loss : 47.525795
epoch = 312 train_loss : 35.653507 , test loss : 47.404415
epoch = 314 train_loss : 35.578346 , test loss : 47.333317
epoch = 316 train_loss : 35.503426 , test loss : 47.298374
epoch = 317 train_loss : 35.466793 , test loss : 47.2121

epoch = 567 train_loss : 30.054209 , test loss : 42.085987
epoch = 570 train_loss : 30.020315 , test loss : 42.010445
epoch = 576 train_loss : 29.937218 , test loss : 42.005051
epoch = 577 train_loss : 29.923521 , test loss : 41.993050
epoch = 581 train_loss : 29.873877 , test loss : 41.954056
epoch = 586 train_loss : 29.813467 , test loss : 41.892212
epoch = 587 train_loss : 29.805359 , test loss : 41.850414
epoch = 592 train_loss : 29.743282 , test loss : 41.838188
epoch = 594 train_loss : 29.716410 , test loss : 41.796318
epoch = 595 train_loss : 29.707144 , test loss : 41.793438
epoch = 599 train_loss : 29.652184 , test loss : 41.783772
epoch = 604 train_loss : 29.591785 , test loss : 41.778481
epoch = 606 train_loss : 29.567490 , test loss : 41.760612
epoch = 607 train_loss : 29.559082 , test loss : 41.680176
epoch = 612 train_loss : 29.500740 , test loss : 41.648155
epoch = 616 train_loss : 29.452074 , test loss : 41.634861
epoch = 618 train_loss : 29.430071 , test loss : 41.6059

epoch = 56 train_loss : 70.372787 , test loss : 115.627823
epoch = 57 train_loss : 69.772804 , test loss : 114.527458
epoch = 58 train_loss : 69.185127 , test loss : 113.514725
epoch = 59 train_loss : 68.611740 , test loss : 112.615997
epoch = 60 train_loss : 68.061012 , test loss : 111.768631
epoch = 61 train_loss : 67.529579 , test loss : 110.945679
epoch = 62 train_loss : 67.016342 , test loss : 110.104019
epoch = 63 train_loss : 66.507683 , test loss : 109.346519
epoch = 64 train_loss : 66.019676 , test loss : 108.576797
epoch = 65 train_loss : 65.542374 , test loss : 107.816940
epoch = 66 train_loss : 65.080620 , test loss : 107.102547
epoch = 67 train_loss : 64.627174 , test loss : 106.370697
epoch = 68 train_loss : 64.192032 , test loss : 105.721283
epoch = 69 train_loss : 63.762009 , test loss : 104.983521
epoch = 70 train_loss : 63.344776 , test loss : 104.436020
epoch = 71 train_loss : 62.940784 , test loss : 103.716690
epoch = 72 train_loss : 62.546871 , test loss : 103.0742

...

epoch = 68 train_loss : 72.496468 , test loss : 42.608692
epoch = 69 train_loss : 71.910118 , test loss : 42.223728
epoch = 70 train_loss : 71.328720 , test loss : 41.855042
epoch = 71 train_loss : 70.768532 , test loss : 41.467857
epoch = 72 train_loss : 70.223419 , test loss : 41.050686
epoch = 73 train_loss : 69.690102 , test loss : 40.732761
epoch = 74 train_loss : 69.163284 , test loss : 40.461094
epoch = 75 train_loss : 68.662506 , test loss : 40.188126
epoch = 76 train_loss : 68.168076 , test loss : 39.842609
epoch = 77 train_loss : 67.684784 , test loss : 39.528637
epoch = 78 train_loss : 67.220520 , test loss : 39.243816
epoch = 79 train_loss : 66.767326 , test loss : 38.968632
epoch = 80 train_loss : 66.316338 , test loss : 38.790543
epoch = 81 train_loss : 65.881775 , test loss : 38.555138
epoch = 82 train_loss : 65.457893 , test loss : 38.362648
epoch = 83 train_loss : 65.050697 , test loss : 38.214241
epoch = 84 train_loss : 64.648239 , test loss : 37.941158
epoch = 85 tra

epoch = 239 train_loss : 41.579098 , test loss : 28.929884
epoch = 244 train_loss : 41.282272 , test loss : 28.902500
epoch = 245 train_loss : 41.229443 , test loss : 28.787560
epoch = 248 train_loss : 41.066601 , test loss : 28.785845
epoch = 250 train_loss : 40.955276 , test loss : 28.783823
epoch = 253 train_loss : 40.799351 , test loss : 28.743238
epoch = 254 train_loss : 40.749744 , test loss : 28.711208
epoch = 255 train_loss : 40.695335 , test loss : 28.696041
epoch = 256 train_loss : 40.650597 , test loss : 28.683599
epoch = 257 train_loss : 40.594780 , test loss : 28.589905
epoch = 265 train_loss : 40.203865 , test loss : 28.543013
epoch = 266 train_loss : 40.155010 , test loss : 28.511171
epoch = 268 train_loss : 40.063126 , test loss : 28.497900
epoch = 271 train_loss : 39.926765 , test loss : 28.470434
epoch = 272 train_loss : 39.890488 , test loss : 28.448076
epoch = 274 train_loss : 39.789959 , test loss : 28.442427
epoch = 275 train_loss : 39.747322 , test loss : 28.4077

epoch = 52 train_loss : 75.693764 , test loss : 102.349869
epoch = 53 train_loss : 74.899673 , test loss : 101.465492
epoch = 54 train_loss : 74.142555 , test loss : 100.663544
epoch = 55 train_loss : 73.396194 , test loss : 99.847893
epoch = 56 train_loss : 72.680466 , test loss : 99.051697
epoch = 57 train_loss : 71.988533 , test loss : 98.270920
epoch = 58 train_loss : 71.319672 , test loss : 97.498589
epoch = 59 train_loss : 70.666481 , test loss : 96.756927
epoch = 60 train_loss : 70.035446 , test loss : 96.013985
epoch = 61 train_loss : 69.424339 , test loss : 95.250191
epoch = 62 train_loss : 68.827766 , test loss : 94.538643
epoch = 63 train_loss : 68.256157 , test loss : 93.845589
epoch = 64 train_loss : 67.695038 , test loss : 93.175499
epoch = 65 train_loss : 67.157669 , test loss : 92.512901
epoch = 66 train_loss : 66.630470 , test loss : 91.885628
epoch = 67 train_loss : 66.124809 , test loss : 91.251648
epoch = 68 train_loss : 65.623978 , test loss : 90.692642
epoch = 69 

epoch = 193 train_loss : 42.211452 , test loss : 58.502296
epoch = 194 train_loss : 42.132713 , test loss : 58.358463
epoch = 195 train_loss : 42.057415 , test loss : 58.251640
epoch = 196 train_loss : 41.982635 , test loss : 58.108696
epoch = 197 train_loss : 41.897438 , test loss : 58.051437
epoch = 198 train_loss : 41.819820 , test loss : 58.003136
epoch = 199 train_loss : 41.744484 , test loss : 57.942909
epoch = 200 train_loss : 41.668770 , test loss : 57.827984
epoch = 201 train_loss : 41.594658 , test loss : 57.754959
epoch = 202 train_loss : 41.521748 , test loss : 57.677971
epoch = 203 train_loss : 41.448952 , test loss : 57.583874
epoch = 204 train_loss : 41.380627 , test loss : 57.565441
epoch = 205 train_loss : 41.304966 , test loss : 57.468758
epoch = 206 train_loss : 41.233540 , test loss : 57.397919
epoch = 207 train_loss : 41.164219 , test loss : 57.296204
epoch = 208 train_loss : 41.093533 , test loss : 57.193241
epoch = 211 train_loss : 40.885193 , test loss : 57.0399

epoch = 426 train_loss : 33.045483 , test loss : 49.682377
epoch = 427 train_loss : 33.025005 , test loss : 49.677586
epoch = 428 train_loss : 33.009590 , test loss : 49.643414
epoch = 431 train_loss : 32.939960 , test loss : 49.625965
epoch = 436 train_loss : 32.838615 , test loss : 49.607231
epoch = 437 train_loss : 32.818195 , test loss : 49.585922
epoch = 441 train_loss : 32.740231 , test loss : 49.529137
epoch = 444 train_loss : 32.678387 , test loss : 49.466621
epoch = 445 train_loss : 32.670368 , test loss : 49.400852
epoch = 450 train_loss : 32.565033 , test loss : 49.367996
epoch = 452 train_loss : 32.521904 , test loss : 49.359627
epoch = 454 train_loss : 32.483994 , test loss : 49.340538
epoch = 456 train_loss : 32.449913 , test loss : 49.274105
epoch = 457 train_loss : 32.429447 , test loss : 49.273914
epoch = 461 train_loss : 32.353886 , test loss : 49.232033
epoch = 462 train_loss : 32.341015 , test loss : 49.211250
epoch = 464 train_loss : 32.297047 , test loss : 49.2061

In [54]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],32),nn.ReLU(),nn.Linear(32,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 326.401733 , test loss : 446.558044
epoch = 2 train_loss : 174.507614 , test loss : 223.570343
epoch = 3 train_loss : 115.756355 , test loss : 160.215942
epoch = 4 train_loss : 92.262581 , test loss : 121.958763
epoch = 5 train_loss : 80.813515 , test loss : 105.771835
epoch = 6 train_loss : 72.826317 , test loss : 96.873482
epoch = 7 train_loss : 66.830727 , test loss : 89.834625
epoch = 8 train_loss : 62.141766 , test loss : 81.167984
epoch = 9 train_loss : 58.513714 , test loss : 74.794250
epoch = 10 train_loss : 55.643913 , test loss : 70.566078
epoch = 11 train_loss : 53.180683 , test loss : 66.707405
epoch = 12 train_loss : 51.221664 , test loss : 65.640656
epoch = 13 train_loss : 49.477413 , test loss : 62.470646
epoch = 14 train_loss : 48.091625 , test loss : 60.704494
epoch = 15 train_loss : 46.914661 , test loss : 58.900822
epoch = 16 train_loss : 45.597763 , test loss : 57.892128
epoch = 17 train_loss : 44.556290 , test loss : 57.094574
epoch = 18 trai

epoch = 28 train_loss : 35.462646 , test loss : 58.390289
epoch = 29 train_loss : 34.618938 , test loss : 58.302814
epoch = 30 train_loss : 34.589146 , test loss : 56.164555
epoch = 33 train_loss : 33.222450 , test loss : 54.910366
epoch = 34 train_loss : 32.973873 , test loss : 53.971619
epoch = 37 train_loss : 32.206451 , test loss : 53.367249
epoch = 39 train_loss : 31.792795 , test loss : 52.344398
epoch = 42 train_loss : 31.091722 , test loss : 52.050091
epoch = 43 train_loss : 31.502491 , test loss : 51.122253
epoch = 45 train_loss : 30.967365 , test loss : 51.047588
epoch = 47 train_loss : 30.465071 , test loss : 50.842651
epoch = 48 train_loss : 30.320601 , test loss : 50.044571
epoch = 56 train_loss : 29.481236 , test loss : 49.926205
epoch = 59 train_loss : 30.364693 , test loss : 49.876747
epoch = 60 train_loss : 28.862150 , test loss : 49.602024
epoch = 63 train_loss : 29.405888 , test loss : 48.340862
epoch = 67 train_loss : 28.595863 , test loss : 48.161873
epoch = 73 tra

In [55]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],16),nn.ReLU(),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 290.644226 , test loss : 416.685822
epoch = 2 train_loss : 181.090347 , test loss : 258.715576
epoch = 3 train_loss : 123.158417 , test loss : 187.100662
epoch = 4 train_loss : 100.855515 , test loss : 144.995331
epoch = 5 train_loss : 88.320129 , test loss : 132.022812
epoch = 6 train_loss : 79.014427 , test loss : 110.217201
epoch = 7 train_loss : 72.803024 , test loss : 97.144547
epoch = 8 train_loss : 66.941017 , test loss : 92.591827
epoch = 9 train_loss : 62.540722 , test loss : 83.660454
epoch = 10 train_loss : 58.985367 , test loss : 79.752708
epoch = 11 train_loss : 55.920765 , test loss : 75.697273
epoch = 12 train_loss : 53.576725 , test loss : 71.944183
epoch = 13 train_loss : 51.520374 , test loss : 68.337013
epoch = 14 train_loss : 49.980522 , test loss : 64.550705
epoch = 16 train_loss : 46.808990 , test loss : 62.166248
epoch = 17 train_loss : 45.399277 , test loss : 60.200581
epoch = 18 train_loss : 44.307873 , test loss : 58.974079
epoch = 19 tr

epoch = 25 train_loss : 36.628998 , test loss : 59.394608
epoch = 26 train_loss : 36.055019 , test loss : 59.124649
epoch = 27 train_loss : 35.532761 , test loss : 58.942913
epoch = 28 train_loss : 35.472691 , test loss : 57.327915
epoch = 30 train_loss : 34.225262 , test loss : 56.605995
epoch = 31 train_loss : 34.261181 , test loss : 56.180557
epoch = 32 train_loss : 33.597130 , test loss : 55.918457
epoch = 33 train_loss : 33.465698 , test loss : 55.695709
epoch = 34 train_loss : 33.002235 , test loss : 55.532467
epoch = 35 train_loss : 32.642010 , test loss : 55.020081
epoch = 37 train_loss : 32.282104 , test loss : 54.139565
epoch = 38 train_loss : 32.272648 , test loss : 54.042576
epoch = 39 train_loss : 32.004513 , test loss : 52.872707
epoch = 41 train_loss : 31.656590 , test loss : 52.281425
epoch = 42 train_loss : 31.300617 , test loss : 52.261593
epoch = 43 train_loss : 31.115410 , test loss : 51.774994
epoch = 47 train_loss : 30.422445 , test loss : 51.302811
epoch = 50 tra

In [56]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],16),nn.Linear(16,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 484.505737 , test loss : 768.815613
epoch = 2 train_loss : 300.815552 , test loss : 693.640869
epoch = 3 train_loss : 155.211151 , test loss : 435.432922
epoch = 4 train_loss : 79.878342 , test loss : 187.748047
epoch = 5 train_loss : 55.615795 , test loss : 90.998306
epoch = 6 train_loss : 48.412884 , test loss : 66.800285
epoch = 7 train_loss : 44.555012 , test loss : 56.638653
epoch = 8 train_loss : 42.286427 , test loss : 52.044258
epoch = 9 train_loss : 40.863125 , test loss : 50.065128
epoch = 10 train_loss : 39.983730 , test loss : 50.008495
epoch = 11 train_loss : 38.709011 , test loss : 46.871059
epoch = 12 train_loss : 38.283154 , test loss : 45.831474
epoch = 13 train_loss : 38.147385 , test loss : 45.499561
epoch = 14 train_loss : 37.941216 , test loss : 45.146942
epoch = 15 train_loss : 36.963463 , test loss : 44.848892
epoch = 17 train_loss : 35.879154 , test loss : 43.045883
epoch = 18 train_loss : 36.159931 , test loss : 42.518593
epoch = 20 train

epoch = 9 train_loss : 42.729988 , test loss : 47.237553
epoch = 10 train_loss : 41.027241 , test loss : 44.423092
epoch = 11 train_loss : 40.327183 , test loss : 43.613594
epoch = 12 train_loss : 39.256428 , test loss : 41.810780
epoch = 15 train_loss : 38.283489 , test loss : 40.349308
epoch = 16 train_loss : 37.420223 , test loss : 39.350170
epoch = 18 train_loss : 41.260815 , test loss : 39.098293
epoch = 19 train_loss : 37.105671 , test loss : 37.500320
epoch = 21 train_loss : 36.333897 , test loss : 36.746979
epoch = 26 train_loss : 35.492420 , test loss : 35.882370
epoch = 28 train_loss : 36.033585 , test loss : 34.897934
epoch = 29 train_loss : 35.293148 , test loss : 34.891781
epoch = 33 train_loss : 36.639729 , test loss : 34.739265
epoch = 35 train_loss : 34.688644 , test loss : 34.324947
epoch = 37 train_loss : 34.492943 , test loss : 33.984501
epoch = 39 train_loss : 34.365154 , test loss : 33.947956
epoch = 41 train_loss : 34.361637 , test loss : 32.800888
epoch = 44 trai

In [57]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],32),nn.Linear(32,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 446.704132 , test loss : 749.628784
epoch = 2 train_loss : 270.282410 , test loss : 641.907471
epoch = 3 train_loss : 134.523376 , test loss : 361.663483
epoch = 4 train_loss : 71.439255 , test loss : 143.233276
epoch = 5 train_loss : 52.260944 , test loss : 78.705452
epoch = 6 train_loss : 46.633137 , test loss : 62.363617
epoch = 7 train_loss : 43.009811 , test loss : 53.498901
epoch = 8 train_loss : 41.340519 , test loss : 50.777958
epoch = 9 train_loss : 40.187443 , test loss : 48.985569
epoch = 10 train_loss : 39.341541 , test loss : 47.339546
epoch = 12 train_loss : 38.217434 , test loss : 46.734585
epoch = 13 train_loss : 37.211868 , test loss : 44.585838
epoch = 18 train_loss : 36.340618 , test loss : 43.359146
epoch = 19 train_loss : 35.643059 , test loss : 41.753296
epoch = 28 train_loss : 33.202793 , test loss : 41.223282
epoch = 31 train_loss : 33.101353 , test loss : 41.177025
epoch = 36 train_loss : 32.618256 , test loss : 40.815979
epoch = 37 train

epoch = 10 train_loss : 41.366882 , test loss : 43.478092
epoch = 11 train_loss : 39.913124 , test loss : 42.664688
epoch = 12 train_loss : 39.839584 , test loss : 41.471901
epoch = 13 train_loss : 38.620689 , test loss : 41.005795
epoch = 15 train_loss : 37.809093 , test loss : 39.683048
epoch = 16 train_loss : 37.696217 , test loss : 38.468464
epoch = 19 train_loss : 36.526512 , test loss : 36.945847
epoch = 23 train_loss : 35.843517 , test loss : 36.142090
epoch = 24 train_loss : 35.744274 , test loss : 36.018612
epoch = 26 train_loss : 35.490448 , test loss : 35.585072
epoch = 27 train_loss : 36.057693 , test loss : 35.066296
epoch = 29 train_loss : 35.264606 , test loss : 34.839584
epoch = 32 train_loss : 34.793758 , test loss : 34.398613
epoch = 34 train_loss : 35.289772 , test loss : 34.221687
epoch = 38 train_loss : 34.488914 , test loss : 34.094963
epoch = 40 train_loss : 34.600315 , test loss : 33.429844
epoch = 42 train_loss : 34.550797 , test loss : 33.391495
epoch = 43 tra

In [58]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],128),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 448.002228 , test loss : 741.289246
epoch = 2 train_loss : 266.878723 , test loss : 642.567749
epoch = 3 train_loss : 131.925369 , test loss : 369.105591
epoch = 4 train_loss : 70.635735 , test loss : 139.366196
epoch = 5 train_loss : 55.092491 , test loss : 83.271492
epoch = 6 train_loss : 47.395672 , test loss : 63.826572
epoch = 7 train_loss : 44.838715 , test loss : 52.680119
epoch = 8 train_loss : 41.659576 , test loss : 51.980816
epoch = 9 train_loss : 40.529327 , test loss : 48.923965
epoch = 10 train_loss : 39.498238 , test loss : 48.274540
epoch = 11 train_loss : 38.599957 , test loss : 47.309433
epoch = 13 train_loss : 37.549927 , test loss : 46.222927
epoch = 14 train_loss : 36.916237 , test loss : 45.139488
epoch = 15 train_loss : 37.274525 , test loss : 44.784473
epoch = 18 train_loss : 35.162788 , test loss : 43.964878
epoch = 21 train_loss : 35.371246 , test loss : 43.482140
epoch = 23 train_loss : 34.550926 , test loss : 42.859295
epoch = 24 train

epoch = 4 train_loss : 61.062542 , test loss : 76.497322
epoch = 5 train_loss : 50.398258 , test loss : 58.546009
epoch = 6 train_loss : 46.341778 , test loss : 51.865303
epoch = 7 train_loss : 43.894539 , test loss : 48.424885
epoch = 8 train_loss : 44.538845 , test loss : 46.059395
epoch = 9 train_loss : 41.467911 , test loss : 45.016727
epoch = 10 train_loss : 40.984287 , test loss : 44.531769
epoch = 11 train_loss : 39.905796 , test loss : 43.007683
epoch = 12 train_loss : 39.529369 , test loss : 41.603237
epoch = 14 train_loss : 38.737747 , test loss : 40.340218
epoch = 15 train_loss : 38.752686 , test loss : 40.125751
epoch = 16 train_loss : 38.751511 , test loss : 39.538486
epoch = 18 train_loss : 36.837685 , test loss : 37.706898
epoch = 19 train_loss : 36.764206 , test loss : 37.546299
epoch = 22 train_loss : 35.897594 , test loss : 36.214409
epoch = 24 train_loss : 35.721123 , test loss : 36.082664
epoch = 25 train_loss : 35.324028 , test loss : 35.340809
epoch = 29 train_los

In [59]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],256),nn.Linear(256,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 411.512146 , test loss : 748.604675
epoch = 2 train_loss : 232.245224 , test loss : 603.912415
epoch = 3 train_loss : 112.273842 , test loss : 293.507538
epoch = 4 train_loss : 64.706696 , test loss : 123.895782
epoch = 5 train_loss : 51.544182 , test loss : 77.814461
epoch = 6 train_loss : 45.833153 , test loss : 56.623127
epoch = 8 train_loss : 42.792183 , test loss : 50.127533
epoch = 9 train_loss : 40.648373 , test loss : 47.594868
epoch = 10 train_loss : 39.615662 , test loss : 47.496109
epoch = 11 train_loss : 39.084023 , test loss : 46.576527
epoch = 12 train_loss : 38.453114 , test loss : 45.073185
epoch = 14 train_loss : 36.951309 , test loss : 43.893139
epoch = 18 train_loss : 35.137508 , test loss : 43.739178
epoch = 19 train_loss : 35.674992 , test loss : 42.579567
epoch = 20 train_loss : 36.429539 , test loss : 42.423065
epoch = 24 train_loss : 34.029427 , test loss : 41.970837
epoch = 29 train_loss : 33.317234 , test loss : 41.273891
epoch = 37 trai

epoch = 11 train_loss : 40.251568 , test loss : 42.368309
epoch = 12 train_loss : 39.883522 , test loss : 42.233536
epoch = 14 train_loss : 38.441925 , test loss : 40.517662
epoch = 16 train_loss : 37.324226 , test loss : 38.193569
epoch = 19 train_loss : 37.036774 , test loss : 37.010414
epoch = 23 train_loss : 36.810474 , test loss : 36.518070
epoch = 24 train_loss : 35.663883 , test loss : 36.001209
epoch = 27 train_loss : 35.158722 , test loss : 35.212734
epoch = 30 train_loss : 35.811489 , test loss : 34.060127
epoch = 38 train_loss : 34.914143 , test loss : 33.496574
epoch = 42 train_loss : 34.172352 , test loss : 33.321201
epoch = 44 train_loss : 34.086021 , test loss : 33.060677
epoch = 50 train_loss : 33.859886 , test loss : 32.590878
epoch = 60 train_loss : 33.811104 , test loss : 32.235142
epoch = 81 train_loss : 33.594452 , test loss : 32.172112
epoch = 82 train_loss : 33.634171 , test loss : 32.087322
epoch = 101 train_loss : 33.793606 , test loss : 32.056133
epoch = 103 t

In [60]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],1024),nn.Linear(1024,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 356.619965 , test loss : 717.998413
epoch = 2 train_loss : 179.545975 , test loss : 459.690033
epoch = 3 train_loss : 85.790077 , test loss : 221.739304
epoch = 4 train_loss : 55.538155 , test loss : 87.805573
epoch = 5 train_loss : 48.092106 , test loss : 63.495861
epoch = 6 train_loss : 44.305580 , test loss : 53.706959
epoch = 7 train_loss : 43.792419 , test loss : 51.037643
epoch = 8 train_loss : 41.437180 , test loss : 50.957886
epoch = 10 train_loss : 39.775520 , test loss : 48.211124
epoch = 11 train_loss : 38.356934 , test loss : 46.198669
epoch = 12 train_loss : 37.305748 , test loss : 45.289097
epoch = 15 train_loss : 37.008301 , test loss : 45.019886
epoch = 16 train_loss : 35.963554 , test loss : 43.307316
epoch = 18 train_loss : 35.050869 , test loss : 42.308449
epoch = 23 train_loss : 34.161621 , test loss : 42.157398
epoch = 24 train_loss : 34.116264 , test loss : 41.658867
epoch = 30 train_loss : 34.523094 , test loss : 41.187767
epoch = 31 train_

In [63]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],256),nn.Linear(256,128),nn.Linear(128,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 287.832184 , test loss : 644.604187
epoch = 2 train_loss : 67.226395 , test loss : 99.935249
epoch = 3 train_loss : 58.119614 , test loss : 68.557732
epoch = 4 train_loss : 44.690708 , test loss : 48.700565
epoch = 7 train_loss : 40.116772 , test loss : 48.083790
epoch = 9 train_loss : 39.178692 , test loss : 46.203152
epoch = 10 train_loss : 38.481728 , test loss : 44.361462
epoch = 13 train_loss : 36.413807 , test loss : 43.524223
epoch = 18 train_loss : 34.978249 , test loss : 43.146637
epoch = 20 train_loss : 34.272629 , test loss : 43.062054
epoch = 25 train_loss : 34.975544 , test loss : 40.996845
epoch = 37 train_loss : 35.933968 , test loss : 40.906975
epoch = 38 train_loss : 32.653225 , test loss : 39.767693
epoch = 59 train_loss : 32.060104 , test loss : 39.532921
epoch = 86 train_loss : 32.323006 , test loss : 39.403278
epoch = 96 train_loss : 31.330593 , test loss : 39.272835
epoch = 115 train_loss : 31.631144 , test loss : 39.252708
epoch = 161 train

In [77]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],256),nn.Linear(256,128),nn.ReLU(),nn.Linear(128,64),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 230.857773 , test loss : 290.818207
epoch = 2 train_loss : 87.753555 , test loss : 112.980804
epoch = 3 train_loss : 61.705654 , test loss : 79.091026
epoch = 5 train_loss : 49.980476 , test loss : 58.531570
epoch = 8 train_loss : 42.737492 , test loss : 49.545815
epoch = 9 train_loss : 39.117123 , test loss : 48.259903
epoch = 10 train_loss : 40.368076 , test loss : 47.810406
epoch = 12 train_loss : 36.048119 , test loss : 46.676247
epoch = 14 train_loss : 35.989353 , test loss : 46.559345
epoch = 16 train_loss : 35.783764 , test loss : 45.180435
epoch = 18 train_loss : 35.108181 , test loss : 44.421383
epoch = 20 train_loss : 32.534042 , test loss : 43.480953
epoch = 22 train_loss : 33.719032 , test loss : 43.019585
epoch = 27 train_loss : 29.866327 , test loss : 40.727161
epoch = 51 train_loss : 26.224174 , test loss : 40.700382
epoch = 62 train_loss : 25.233822 , test loss : 40.222454
------------------------------------------------------------
fold 1,train l

In [85]:
arr1=np.array([1,2,3,8,0])
arr2=np.array([4,5,6])

In [84]:
np.vstack((arr1,arr2))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [86]:
np.hstack((arr1,arr2))

array([1, 2, 3, 8, 0, 4, 5, 6])

In [87]:
674+2634

3308

In [115]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],10),nn.ReLU(),nn.Linear(10,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 423.788666 , test loss : 638.793457
epoch = 2 train_loss : 213.760590 , test loss : 310.352936
epoch = 3 train_loss : 155.049927 , test loss : 262.294769
epoch = 4 train_loss : 123.122963 , test loss : 207.725891
epoch = 5 train_loss : 104.549316 , test loss : 185.198532
epoch = 6 train_loss : 91.003143 , test loss : 154.156937
epoch = 7 train_loss : 81.165916 , test loss : 131.192825
epoch = 8 train_loss : 73.422462 , test loss : 120.766220
epoch = 9 train_loss : 67.377129 , test loss : 104.801842
epoch = 10 train_loss : 62.759506 , test loss : 98.375191
epoch = 11 train_loss : 58.752281 , test loss : 89.808052
epoch = 12 train_loss : 55.886147 , test loss : 85.019989
epoch = 13 train_loss : 53.032124 , test loss : 77.890022
epoch = 14 train_loss : 50.747086 , test loss : 73.629387
epoch = 15 train_loss : 50.516064 , test loss : 69.407883
epoch = 16 train_loss : 47.504204 , test loss : 69.107529
epoch = 17 train_loss : 46.087875 , test loss : 65.131027
epoch = 1

epoch = 41 train_loss : 31.659275 , test loss : 51.323803
epoch = 42 train_loss : 31.526512 , test loss : 50.230606
epoch = 47 train_loss : 30.409794 , test loss : 49.756184
epoch = 49 train_loss : 30.396860 , test loss : 49.084293
epoch = 50 train_loss : 30.147352 , test loss : 48.763313
epoch = 53 train_loss : 29.784502 , test loss : 48.474865
epoch = 58 train_loss : 29.663527 , test loss : 48.235226
epoch = 62 train_loss : 29.633236 , test loss : 47.943851
epoch = 64 train_loss : 28.974762 , test loss : 47.540478
epoch = 67 train_loss : 29.363058 , test loss : 47.279030
epoch = 71 train_loss : 29.133757 , test loss : 46.933079
epoch = 75 train_loss : 28.390158 , test loss : 46.596962
epoch = 79 train_loss : 28.217369 , test loss : 46.580833
epoch = 80 train_loss : 28.040670 , test loss : 46.553123
epoch = 88 train_loss : 30.362207 , test loss : 46.548607
epoch = 91 train_loss : 27.623726 , test loss : 46.067917
epoch = 107 train_loss : 27.690161 , test loss : 45.810837
epoch = 131 t

In [116]:
net1=nn.Sequential(nn.Linear(x2.shape[1],4),nn.Linear(4,1))
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],4),nn.Linear(4,1))
net1=get_net()
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 487.730652 , test loss : 770.339111
epoch = 2 train_loss : 305.008240 , test loss : 693.865906
epoch = 3 train_loss : 162.654190 , test loss : 439.623749
epoch = 4 train_loss : 84.653076 , test loss : 190.714783
epoch = 5 train_loss : 55.010036 , test loss : 86.650375
epoch = 6 train_loss : 47.617416 , test loss : 60.025074
epoch = 7 train_loss : 44.566147 , test loss : 58.115513
epoch = 8 train_loss : 42.377804 , test loss : 50.345234
epoch = 10 train_loss : 39.978363 , test loss : 49.053478
epoch = 12 train_loss : 38.272732 , test loss : 47.170101
epoch = 13 train_loss : 37.955215 , test loss : 46.841038
epoch = 14 train_loss : 37.590221 , test loss : 46.263748
epoch = 15 train_loss : 36.809273 , test loss : 45.362679
epoch = 16 train_loss : 37.244896 , test loss : 44.198448
epoch = 17 train_loss : 35.616879 , test loss : 42.574902
epoch = 25 train_loss : 34.710903 , test loss : 42.381130
epoch = 26 train_loss : 33.803268 , test loss : 41.698593
epoch = 28 trai

epoch = 17 train_loss : 37.768021 , test loss : 39.764442
epoch = 18 train_loss : 36.992836 , test loss : 38.580086
epoch = 20 train_loss : 36.930454 , test loss : 37.828323
epoch = 22 train_loss : 37.648724 , test loss : 37.007782
epoch = 23 train_loss : 36.133537 , test loss : 36.701263
epoch = 24 train_loss : 36.377876 , test loss : 36.119061
epoch = 26 train_loss : 35.566406 , test loss : 35.156696
epoch = 31 train_loss : 35.636086 , test loss : 35.002892
epoch = 33 train_loss : 36.104279 , test loss : 34.501923
epoch = 34 train_loss : 34.574913 , test loss : 34.381378
epoch = 35 train_loss : 34.642235 , test loss : 33.905647
epoch = 38 train_loss : 34.621983 , test loss : 33.837559
epoch = 45 train_loss : 34.054039 , test loss : 33.835346
epoch = 49 train_loss : 35.071255 , test loss : 33.757484
epoch = 50 train_loss : 34.803619 , test loss : 32.932079
epoch = 51 train_loss : 33.963379 , test loss : 32.701511
epoch = 54 train_loss : 34.253670 , test loss : 32.683624
epoch = 56 tra

In [117]:
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,1))
net1=get_net()
train_kfold(net1,2000,0.0001,5,x2,y2,256,0.8,0.05)

epoch = 1 train_loss : 549.458008 , test loss : 807.204651
epoch = 2 train_loss : 286.710297 , test loss : 412.620361
epoch = 3 train_loss : 178.982620 , test loss : 227.740891
epoch = 4 train_loss : 138.258133 , test loss : 183.614685
epoch = 5 train_loss : 114.165939 , test loss : 148.013626
epoch = 6 train_loss : 101.046837 , test loss : 127.767570
epoch = 7 train_loss : 92.844070 , test loss : 116.044640
epoch = 8 train_loss : 86.597549 , test loss : 107.208389
epoch = 9 train_loss : 81.472427 , test loss : 100.852974
epoch = 10 train_loss : 77.185196 , test loss : 94.591606
epoch = 11 train_loss : 73.438576 , test loss : 91.278458
epoch = 12 train_loss : 70.251495 , test loss : 86.959160
epoch = 13 train_loss : 67.491684 , test loss : 84.659889
epoch = 14 train_loss : 65.055122 , test loss : 80.710381
epoch = 15 train_loss : 63.029472 , test loss : 77.984100
epoch = 16 train_loss : 61.143806 , test loss : 76.232750
epoch = 17 train_loss : 59.511837 , test loss : 74.162407
epoch = 

epoch = 25 train_loss : 49.879608 , test loss : 82.591316
epoch = 26 train_loss : 49.109005 , test loss : 81.562515
epoch = 27 train_loss : 48.444798 , test loss : 80.697479
epoch = 28 train_loss : 47.763134 , test loss : 79.327904
epoch = 29 train_loss : 47.143719 , test loss : 78.291359
epoch = 30 train_loss : 46.576172 , test loss : 77.667130
epoch = 31 train_loss : 46.027245 , test loss : 76.550255
epoch = 32 train_loss : 45.484505 , test loss : 75.697250
epoch = 33 train_loss : 44.978176 , test loss : 75.019913
epoch = 34 train_loss : 44.477711 , test loss : 74.059944
epoch = 35 train_loss : 44.036835 , test loss : 73.525009
epoch = 36 train_loss : 43.590557 , test loss : 72.766235
epoch = 37 train_loss : 43.172554 , test loss : 72.026596
epoch = 38 train_loss : 42.760571 , test loss : 71.414146
epoch = 39 train_loss : 42.388432 , test loss : 70.816170
epoch = 40 train_loss : 42.000122 , test loss : 70.275620
epoch = 41 train_loss : 41.647331 , test loss : 69.722527
epoch = 42 tra

epoch = 318 train_loss : 23.246342 , test loss : 47.903893
epoch = 342 train_loss : 22.762037 , test loss : 47.903004
epoch = 378 train_loss : 22.064428 , test loss : 47.804516
------------------------------------------------------------
fold 2,train loss mean : 22.064428,test loss : 48.041977
------------------------------------------------------------
epoch = 1 train_loss : 617.449463 , test loss : 419.417999
epoch = 2 train_loss : 225.511185 , test loss : 293.251160
epoch = 3 train_loss : 158.115402 , test loss : 254.243713
epoch = 4 train_loss : 124.694878 , test loss : 224.730942
epoch = 5 train_loss : 109.613594 , test loss : 198.579269
epoch = 6 train_loss : 99.589630 , test loss : 174.450012
epoch = 7 train_loss : 91.563011 , test loss : 156.991028
epoch = 8 train_loss : 85.002190 , test loss : 141.665329
epoch = 9 train_loss : 79.699867 , test loss : 130.061417
epoch = 10 train_loss : 75.318512 , test loss : 121.114830
epoch = 11 train_loss : 71.546051 , test loss : 114.421394

epoch = 3 train_loss : 182.922409 , test loss : 142.995880
epoch = 4 train_loss : 143.944733 , test loss : 107.365204
epoch = 5 train_loss : 123.278969 , test loss : 85.257149
epoch = 6 train_loss : 111.010811 , test loss : 72.317390
epoch = 7 train_loss : 101.838562 , test loss : 63.402050
epoch = 8 train_loss : 94.297623 , test loss : 57.249550
epoch = 9 train_loss : 88.331566 , test loss : 51.751476
epoch = 10 train_loss : 83.076965 , test loss : 48.334232
epoch = 11 train_loss : 78.872307 , test loss : 45.465652
epoch = 12 train_loss : 75.388512 , test loss : 43.374077
epoch = 13 train_loss : 72.289276 , test loss : 41.959606
epoch = 14 train_loss : 69.557236 , test loss : 40.519684
epoch = 15 train_loss : 67.233490 , test loss : 39.698826
epoch = 16 train_loss : 65.213890 , test loss : 38.409626
epoch = 17 train_loss : 63.358009 , test loss : 37.605476
epoch = 18 train_loss : 61.729588 , test loss : 37.122955
epoch = 19 train_loss : 60.277824 , test loss : 36.391117
epoch = 20 tra

epoch = 108 train_loss : 31.261408 , test loss : 51.035919
epoch = 111 train_loss : 30.997675 , test loss : 50.922508
epoch = 115 train_loss : 30.776052 , test loss : 50.813427
epoch = 117 train_loss : 30.571974 , test loss : 50.813000
epoch = 119 train_loss : 30.421047 , test loss : 50.761635
epoch = 122 train_loss : 30.242033 , test loss : 50.697712
epoch = 130 train_loss : 29.858934 , test loss : 50.620132
epoch = 141 train_loss : 29.077999 , test loss : 50.583931
------------------------------------------------------------
fold 5,train loss mean : 29.077999,test loss : 50.970863
------------------------------------------------------------
5 fold ,total train loss mean : 24.831504,total test loss mean : 42.757274 
------------------------------------------------------------


In [118]:
# net1=nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,1))
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.00001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 666.292847 , test loss : 967.500916
epoch = 2 train_loss : 665.425354 , test loss : 966.414307
epoch = 3 train_loss : 664.551941 , test loss : 965.320190
epoch = 4 train_loss : 663.672363 , test loss : 964.217896
epoch = 5 train_loss : 662.782959 , test loss : 963.103394
epoch = 6 train_loss : 661.883240 , test loss : 961.975037
epoch = 7 train_loss : 660.969849 , test loss : 960.826599
epoch = 8 train_loss : 660.043152 , test loss : 959.661072
epoch = 9 train_loss : 659.099670 , test loss : 958.472595
epoch = 10 train_loss : 658.136963 , test loss : 957.258240
epoch = 11 train_loss : 657.157898 , test loss : 956.022461
epoch = 12 train_loss : 656.161926 , test loss : 954.762939
epoch = 13 train_loss : 655.143860 , test loss : 953.474182
epoch = 14 train_loss : 654.102356 , test loss : 952.153625
epoch = 15 train_loss : 653.034790 , test loss : 950.797546
epoch = 16 train_loss : 651.938538 , test loss : 949.403931
epoch = 17 train_loss : 650.813110 , test loss : 

epoch = 140 train_loss : 120.237511 , test loss : 153.282135
epoch = 141 train_loss : 119.349014 , test loss : 151.903595
epoch = 142 train_loss : 118.475868 , test loss : 150.523041
epoch = 143 train_loss : 117.630951 , test loss : 149.203186
epoch = 144 train_loss : 116.804741 , test loss : 147.935852
epoch = 145 train_loss : 116.003838 , test loss : 146.680618
epoch = 146 train_loss : 115.219215 , test loss : 145.501556
epoch = 147 train_loss : 114.454735 , test loss : 144.365234
epoch = 148 train_loss : 113.703362 , test loss : 143.196243
epoch = 149 train_loss : 112.973610 , test loss : 142.062561
epoch = 150 train_loss : 112.264709 , test loss : 140.951111
epoch = 151 train_loss : 111.570686 , test loss : 139.875687
epoch = 152 train_loss : 110.904030 , test loss : 138.853043
epoch = 153 train_loss : 110.246712 , test loss : 137.830368
epoch = 154 train_loss : 109.602112 , test loss : 136.821732
epoch = 155 train_loss : 108.975250 , test loss : 135.854706
epoch = 156 train_loss :

epoch = 279 train_loss : 74.687492 , test loss : 90.412590
epoch = 280 train_loss : 74.537025 , test loss : 90.258873
epoch = 281 train_loss : 74.387764 , test loss : 90.081650
epoch = 282 train_loss : 74.238564 , test loss : 89.911324
epoch = 283 train_loss : 74.090897 , test loss : 89.760345
epoch = 284 train_loss : 73.944855 , test loss : 89.573471
epoch = 285 train_loss : 73.797791 , test loss : 89.417099
epoch = 286 train_loss : 73.652817 , test loss : 89.237823
epoch = 287 train_loss : 73.509285 , test loss : 89.073517
epoch = 288 train_loss : 73.366356 , test loss : 88.906731
epoch = 289 train_loss : 73.223618 , test loss : 88.713356
epoch = 290 train_loss : 73.083122 , test loss : 88.584145
epoch = 291 train_loss : 72.943291 , test loss : 88.478920
epoch = 292 train_loss : 72.803596 , test loss : 88.308563
epoch = 293 train_loss : 72.664360 , test loss : 88.160789
epoch = 294 train_loss : 72.527100 , test loss : 88.001732
epoch = 295 train_loss : 72.390007 , test loss : 87.8531

epoch = 420 train_loss : 59.874153 , test loss : 73.421844
epoch = 421 train_loss : 59.801147 , test loss : 73.331917
epoch = 422 train_loss : 59.728798 , test loss : 73.246979
epoch = 423 train_loss : 59.656151 , test loss : 73.186417
epoch = 424 train_loss : 59.583759 , test loss : 73.119843
epoch = 425 train_loss : 59.510822 , test loss : 73.026634
epoch = 426 train_loss : 59.438686 , test loss : 72.941658
epoch = 427 train_loss : 59.367859 , test loss : 72.850555
epoch = 428 train_loss : 59.296101 , test loss : 72.745949
epoch = 429 train_loss : 59.225780 , test loss : 72.674789
epoch = 430 train_loss : 59.156311 , test loss : 72.599968
epoch = 431 train_loss : 59.086487 , test loss : 72.528091
epoch = 432 train_loss : 59.017529 , test loss : 72.456955
epoch = 433 train_loss : 58.947468 , test loss : 72.369270
epoch = 434 train_loss : 58.879105 , test loss : 72.284225
epoch = 435 train_loss : 58.811142 , test loss : 72.233482
epoch = 436 train_loss : 58.743317 , test loss : 72.1694

epoch = 559 train_loss : 52.030087 , test loss : 64.720291
epoch = 560 train_loss : 51.986641 , test loss : 64.672783
epoch = 561 train_loss : 51.943130 , test loss : 64.642609
epoch = 562 train_loss : 51.899876 , test loss : 64.605301
epoch = 563 train_loss : 51.857178 , test loss : 64.571716
epoch = 564 train_loss : 51.813828 , test loss : 64.511292
epoch = 565 train_loss : 51.770767 , test loss : 64.468346
epoch = 566 train_loss : 51.727745 , test loss : 64.408195
epoch = 567 train_loss : 51.685234 , test loss : 64.367340
epoch = 568 train_loss : 51.642551 , test loss : 64.330460
epoch = 569 train_loss : 51.600132 , test loss : 64.291855
epoch = 570 train_loss : 51.557457 , test loss : 64.239136
epoch = 571 train_loss : 51.515175 , test loss : 64.168915
epoch = 572 train_loss : 51.473110 , test loss : 64.127274
epoch = 573 train_loss : 51.431194 , test loss : 64.065994
epoch = 574 train_loss : 51.389133 , test loss : 64.046463
epoch = 575 train_loss : 51.347416 , test loss : 64.0058

...

epoch = 307 train_loss : 69.119911 , test loss : 98.115448
epoch = 308 train_loss : 69.006821 , test loss : 97.980865
epoch = 309 train_loss : 68.895180 , test loss : 97.839706
epoch = 310 train_loss : 68.784134 , test loss : 97.693756
epoch = 311 train_loss : 68.673500 , test loss : 97.567986
epoch = 312 train_loss : 68.561470 , test loss : 97.438766
epoch = 313 train_loss : 68.452644 , test loss : 97.302620
epoch = 314 train_loss : 68.344429 , test loss : 97.165947
epoch = 315 train_loss : 68.237122 , test loss : 97.027359
epoch = 316 train_loss : 68.129379 , test loss : 96.899956
epoch = 317 train_loss : 68.022087 , test loss : 96.767174
epoch = 318 train_loss : 67.915741 , test loss : 96.633858
epoch = 319 train_loss : 67.809868 , test loss : 96.503853
epoch = 320 train_loss : 67.705093 , test loss : 96.385117
epoch = 321 train_loss : 67.600525 , test loss : 96.244568
epoch = 322 train_loss : 67.497208 , test loss : 96.124077
epoch = 323 train_loss : 67.392624 , test loss : 95.9854

epoch = 446 train_loss : 57.742794 , test loss : 83.312798
epoch = 447 train_loss : 57.683758 , test loss : 83.229431
epoch = 448 train_loss : 57.624355 , test loss : 83.145775
epoch = 449 train_loss : 57.565483 , test loss : 83.068321
epoch = 450 train_loss : 57.506840 , test loss : 82.978554
epoch = 451 train_loss : 57.447788 , test loss : 82.897629
epoch = 452 train_loss : 57.389450 , test loss : 82.812881
epoch = 453 train_loss : 57.331192 , test loss : 82.738312
epoch = 454 train_loss : 57.273098 , test loss : 82.665192
epoch = 455 train_loss : 57.215412 , test loss : 82.580017
epoch = 456 train_loss : 57.157623 , test loss : 82.502617
epoch = 457 train_loss : 57.100338 , test loss : 82.424782
epoch = 458 train_loss : 57.043667 , test loss : 82.344032
epoch = 459 train_loss : 56.986286 , test loss : 82.277672
epoch = 460 train_loss : 56.929131 , test loss : 82.203499
epoch = 461 train_loss : 56.872997 , test loss : 82.124077
epoch = 462 train_loss : 56.816193 , test loss : 82.0491

epoch = 587 train_loss : 51.032753 , test loss : 74.316925
epoch = 588 train_loss : 50.994308 , test loss : 74.272125
epoch = 589 train_loss : 50.955956 , test loss : 74.217987
epoch = 590 train_loss : 50.918056 , test loss : 74.164185
epoch = 591 train_loss : 50.879902 , test loss : 74.114563
epoch = 592 train_loss : 50.842125 , test loss : 74.069077
epoch = 593 train_loss : 50.803955 , test loss : 74.011444
epoch = 594 train_loss : 50.766319 , test loss : 73.954170
epoch = 595 train_loss : 50.728592 , test loss : 73.903069
epoch = 596 train_loss : 50.690830 , test loss : 73.856918
epoch = 597 train_loss : 50.653263 , test loss : 73.801682
epoch = 598 train_loss : 50.615829 , test loss : 73.736862
epoch = 599 train_loss : 50.578751 , test loss : 73.689514
epoch = 600 train_loss : 50.541168 , test loss : 73.647270
epoch = 601 train_loss : 50.503750 , test loss : 73.599106
epoch = 602 train_loss : 50.466145 , test loss : 73.565941
epoch = 603 train_loss : 50.429050 , test loss : 73.5268

epoch = 728 train_loss : 46.374264 , test loss : 68.249977
epoch = 729 train_loss : 46.345821 , test loss : 68.216805
epoch = 730 train_loss : 46.317360 , test loss : 68.191109
epoch = 731 train_loss : 46.289032 , test loss : 68.132309
epoch = 732 train_loss : 46.261063 , test loss : 68.090622
epoch = 733 train_loss : 46.232620 , test loss : 68.055084
epoch = 734 train_loss : 46.204731 , test loss : 68.009148
epoch = 735 train_loss : 46.176590 , test loss : 67.977356
epoch = 736 train_loss : 46.148773 , test loss : 67.944077
epoch = 737 train_loss : 46.120541 , test loss : 67.923790
epoch = 738 train_loss : 46.092705 , test loss : 67.888252
epoch = 739 train_loss : 46.064785 , test loss : 67.853432
epoch = 740 train_loss : 46.036869 , test loss : 67.836670
epoch = 741 train_loss : 46.009384 , test loss : 67.794487
epoch = 742 train_loss : 45.981865 , test loss : 67.739090
epoch = 743 train_loss : 45.953915 , test loss : 67.711075
epoch = 744 train_loss : 45.926128 , test loss : 67.6740

epoch = 867 train_loss : 42.902744 , test loss : 64.036850
epoch = 868 train_loss : 42.880836 , test loss : 64.003922
epoch = 869 train_loss : 42.858761 , test loss : 63.997528
epoch = 870 train_loss : 42.836937 , test loss : 63.972160
epoch = 871 train_loss : 42.815189 , test loss : 63.946339
epoch = 872 train_loss : 42.793514 , test loss : 63.931908
epoch = 873 train_loss : 42.771778 , test loss : 63.902687
epoch = 874 train_loss : 42.750134 , test loss : 63.878887
epoch = 875 train_loss : 42.728558 , test loss : 63.839230
epoch = 876 train_loss : 42.706902 , test loss : 63.808510
epoch = 877 train_loss : 42.685337 , test loss : 63.783028
epoch = 878 train_loss : 42.663906 , test loss : 63.757824
epoch = 879 train_loss : 42.642380 , test loss : 63.733879
epoch = 880 train_loss : 42.620953 , test loss : 63.722775
epoch = 881 train_loss : 42.599617 , test loss : 63.701813
epoch = 882 train_loss : 42.578232 , test loss : 63.673458
epoch = 883 train_loss : 42.557053 , test loss : 63.6379

In [25]:
net1=nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,1))
def get_net():
    return nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,1))
net1=get_net()
train_kfold(net1,1000,0.0001,5,x2,y2,256,0.8,0)

epoch = 1 train_loss : 640.408569 , test loss : 935.648621
epoch = 2 train_loss : 499.265594 , test loss : 743.917480
epoch = 3 train_loss : 164.864868 , test loss : 182.410522
epoch = 4 train_loss : 107.562477 , test loss : 119.404381
epoch = 5 train_loss : 90.673538 , test loss : 98.595352
epoch = 6 train_loss : 79.714195 , test loss : 91.111366
epoch = 7 train_loss : 72.363762 , test loss : 89.047882
epoch = 8 train_loss : 65.993210 , test loss : 78.100937
epoch = 9 train_loss : 61.678204 , test loss : 72.454292
epoch = 10 train_loss : 58.043919 , test loss : 69.410591
epoch = 11 train_loss : 55.164738 , test loss : 66.510902
epoch = 12 train_loss : 52.721596 , test loss : 63.970070
epoch = 13 train_loss : 50.653843 , test loss : 61.807148
epoch = 14 train_loss : 49.052128 , test loss : 59.136318
epoch = 15 train_loss : 47.442951 , test loss : 59.002674
epoch = 16 train_loss : 45.992561 , test loss : 56.136421
epoch = 17 train_loss : 44.695766 , test loss : 55.205200
epoch = 18 trai

epoch = 35 train_loss : 30.491871 , test loss : 55.239754
epoch = 36 train_loss : 30.181654 , test loss : 54.770344
epoch = 37 train_loss : 29.945721 , test loss : 54.417599
epoch = 39 train_loss : 29.889851 , test loss : 53.748779
epoch = 41 train_loss : 28.970600 , test loss : 53.259037
epoch = 44 train_loss : 28.367868 , test loss : 53.061131
epoch = 45 train_loss : 28.048958 , test loss : 52.335991
epoch = 47 train_loss : 27.475012 , test loss : 51.769871
epoch = 50 train_loss : 26.860620 , test loss : 51.481659
epoch = 54 train_loss : 26.216909 , test loss : 51.396210
epoch = 55 train_loss : 26.105568 , test loss : 51.178905
epoch = 56 train_loss : 25.920126 , test loss : 50.849922
epoch = 59 train_loss : 25.618546 , test loss : 50.516125
epoch = 61 train_loss : 25.217686 , test loss : 50.435051
epoch = 62 train_loss : 25.296976 , test loss : 49.870861
epoch = 65 train_loss : 24.724005 , test loss : 49.790131
epoch = 66 train_loss : 24.786507 , test loss : 49.497917
epoch = 76 tra

In [26]:
net1=nn.Sequential(nn.Linear(x2.shape[1],128),nn.ReLU(),nn.Linear(128,64),nn.ReLU(),nn.Linear(64,1))
train_kfold(net1,1000,0.001,5,x2,y2,256,0,0)

epoch = 1 train_loss : 185.634338 , test loss : 243.138977
epoch = 2 train_loss : 100.431992 , test loss : 132.070724
epoch = 3 train_loss : 75.604828 , test loss : 98.485878
epoch = 4 train_loss : 62.990891 , test loss : 74.724411
epoch = 5 train_loss : 60.806892 , test loss : 68.434479
epoch = 6 train_loss : 51.510925 , test loss : 63.760334
epoch = 8 train_loss : 48.611092 , test loss : 57.150375
epoch = 9 train_loss : 43.344215 , test loss : 54.163712
epoch = 10 train_loss : 41.738605 , test loss : 52.934132
epoch = 11 train_loss : 40.008118 , test loss : 51.019173
epoch = 12 train_loss : 38.867516 , test loss : 48.256779
epoch = 14 train_loss : 36.435490 , test loss : 47.043800
epoch = 15 train_loss : 35.761543 , test loss : 46.360039
epoch = 17 train_loss : 33.448238 , test loss : 44.558857
epoch = 18 train_loss : 32.764061 , test loss : 44.520943
epoch = 19 train_loss : 32.973522 , test loss : 44.039917
epoch = 26 train_loss : 29.690075 , test loss : 43.626343
epoch = 27 train_l